# **マーケティングミックスモデリング**
- **MMM:Marketing Mix Modeling**

- この方法は、**本当に売り上げに貢献している広告は、どのドメインの広告か？**を明瞭にする方法。
- **売上と広告媒体との関係性をモデリングし**、どの広告媒体が売り上げにどれだけの貢献、寄与しているかを分析する
- **MMM**は、単純なこれまでの効果を分析するのではなく、今後の推移はどうなるのかという予測分析であり、**扱うデータは時系列データ**
- **利用するデータセット**
    - **Week**：週
    - **Sales**：売上
    - **TVCM**：TV CMのコスト
    - **Newspaper**：新聞の折り込みチラシのコスト
    - **Web**：Web広告のコスト
- **得られる結果**
    - **売上貢献度**
        - **どの広告媒体が、どれだけ売上に貢献したのかを可視化**
        - **売り上げ貢献度を計算：数理モデルを構築**
            - **$Sales = \beta_{0} + \beta_{TVCM} × TVCM + \beta_{Newspaper} × Newspaper + \beta_{Web} × Web + \epsilon$**
            - ⇒　**予測モデルに使用**
    - **ROI =（売上貢献度 － コスト）÷ コスト**
        - **広告媒体のコストデータがあれば、ROIを可視化**
    - **予測モデル**
- **MMM構築パイプライン**
    - 1. **売上を目的変数**、TVCM・Newspaper・WebCostを説明変数にモデルを構築
    - 2. **構築したモデルを使用し**、**媒体別の売り上げ貢献度を求める**
    - 3. 売上貢献度を足し合わせると**売上予測値**になる。この状態では実測値と乖離しているので、**補正係数を計算し、補正済みの売上貢献度を計算する**
    - 4. **補正済みの売り上げ貢献度を使用**し、**媒体別のROIを計算**
    
## [**AdStockモデル(アドストックモデル)**](#AdStock)
- **AdStock**を考慮しないモデルは**Baselineモデル**として規準にする
- アドストック（Ad Stock）を考慮するとは、**飽和モデル(収穫逓減)**　と　**残存モデルを考慮する**
    - **ラグ効果・残差効果・キャリーオーヴァー効果**
    - その日だけに効果があるのではなく、次の日以降もその効果が続くということ
- **飽和効果＋ラグ効果 = AdStock(アドストック)**
    - 効果が遅れてくる、効果が後々まで残っている
        - **ドメイン知**
            - 高額商品、広告などに触れた瞬間購入ではなく、数日～の検討によって購入
            - 消費財等の安価商品、広告に触れるうちに必要になってくる、購入したくなる場合もある。
- **AdStockモデルでの変数の相関の対処**
    - 広告などのマーケティング変数は、**お互いに強い相関を示すことがある**。なぜならば、同じ時期にキャンペーンという名目で広告などを投入したりする
        - これが理由により、線形回帰モデルにとってやっかいな現象が起こることがある
        - **対策**
            - **正則化項付き線形回帰モデルを使用する。**
                - [**Ridge回帰、Lasso回帰、ElasticNet**](#RLE) ⇒ 長項になるのでRidge,Lasso,ElasticNet項で説明
            - **主成分分析回帰モデル(PCR)**
                - [**主成分分析回帰(PCR)**](#PCR) ⇒ 主成分分析回帰項で説明
            - **PLS回帰**
                - [**部分的最小2乗回帰**](#PLS) ⇒ PLS項で説明
- **AdStockモデルでの単純線形回帰モデルの問題点**
    - ある広告にコストをかければかけるほど、**売上の上昇幅は鈍くなる**、経済学でいうところの**収穫逓減**が起こる
    - 売上が飽和し、売上は飽和し、いくらコストをかけても売上が伸びなくなる。
        - AdStockを考慮した線形回帰モデルで目的変数(売上)を予測するとは
            - 広告などの説明変数のデータ（今回の例では、広告などのコスト）を**数値変換（変換器）**
            - **線形回帰モデルの新たなインプットデータ**を作り、その**インプットデータを使い線形回帰モデルで目的変数である売上（Sales）を予測すること**
                - **元の説明変数のデータを数値変換（変換器）** し、このデータを使い線形回帰モデルを構築する。**つまりパイプラインを構築する**
- **AdStockを表現する関数の選定**
    - **最適なAdStockを探索し、モデル構築する**、つまり飽和モデルとラグモデルの最適な組み合わせを探す
    - **解釈**
        - AdStockを表現する関数が、**違和感なく解釈可能なものかどうか**
        - 解釈はドメイン(現場感)が重要
    - **精度**
        - 目的変数Y(例：売上、Sales)の**予測値と実測値が近い値になっているかどうか**というもの
        - 機械的手法で検討する。最も精度の高いものを探索する
- **AdStockモデルの最適な組み合わせ**
    - **2種類のラグ効果モデルから関数を1つ**、**3種類の飽和モデルから関数を1つ選択**する
    - [**ラグモデル**](#lag_model)
    - [**ラグ変換器**](#lag_Converter)
        - **定率減少型ラグモデル(Simple_CarryOver)**
            - ピークが広告などの投入時で**徐々に低減するモデル**
                - $x_{t}^{*}=\displaystyle\sum_{l=0}^{L-1}w_{t-l}・x_{t-l}, w_{t-l}=R^{l}$
                - $x_{t}はt期の広告などの投入量で、x_{x}^{*}はt期とそれ以前までの広告などの効果の累積(残存・ラグ効果を加算蓄積)$
                - **ハイパーパラメータ**
                    - **L(length)**：効果の続く期間 * 当期は含まず
                        - 期間とは、どのくらいまで考慮するか
                    - **R(rate)**：減衰率
        - **ピーク可変型ラグモデル(Peak_CarryOver)**
            - ピークが広告などの投入時に**限らない**
                - $x_{t}^{*}=\frac{\displaystyle\sum_{l=0}^{L-1}w_{t-l}・x_{t-l}}{\displaystyle\sum_{l=0}^{L-1}w_{t-l}},w_{t-l}=R^{(l-P)^{2}}$
                - $x_{t}はt期の広告などの投入量で、x_{x}^{*}はt期とそれ以前までの広告などの効果の累積(残存・ラグ効果を加算蓄積)$
                - **ハイパーパラメータ**
                    - **L(length)**：効果の続く期間　※当期含む
                    - **P(peak)**：ピークの時期（広告などを打った日の場合は0、次期は1、など）
                    - **R(rate)**：減衰率
    - [**飽和モデル**](#Saturation)
    - [**飽和変換器**](#Sat_Converter)
        - **指数型飽和(exp_Saturation)**
            - $x_{t}^{**}=1-e^{-\alpha x_{t}^{*}}$
            - $x_{t}^{*}はt期とそれ以前までの広告などの効果の蓄積(残存・ラグ効果を加算蓄積)で、x_{t}^{**}は指数型飽和モデルで変化した後のt期の値$
        - **ロジスティック型飽和モデル(logit_Saturation)**
            - $x_{t}^{**}=\frac{K}{1+be^{c(c_{t}^{*}-m)}}$
        - **ゴンペルツ型飽和モデル(gom_Saturation)**
            - $x_{t}^{**}=Kb^{e^{-c(x_{t}^{*}-m)}}$
- **AdStockモデルのパイプライン作成の流れ**
    - **入力(変数) ⇒ 変換器(残差・飽和：2つの変換器) ⇒　学習器(Model) ⇒ 出力**
        - [**変換器 : 【1】**](#AdStock_defo)
            - **広告を打った時が効果ピーク、徐々に効果が一定の割合で減退していくモデル**
            - **ラグ(キャリーオーバー)効果モデル**
                - ラグ効果モデルを表現する数理モデルは複数ある。
                - **シンプルな効果モデル：**
                    - **広告などを打ったときが効果のピークで、徐々に効果が一定の割合で減退していくモデル**
                        - $x_{t}^{*}=\displaystyle\sum_{l=0}^{L}w_{t-l}・x_{t-l}, w_{t-l}=R^{l}$
                        - $x_{t}はt期の広告などの投入量で、x_{x}^{*}はt期とそれ以前までの広告などの効果の累積(残存・ラグ効果を加算蓄積)$
                        - **ハイパーパラメータ**
                            - **L(length)**：効果の続く期間 * 当期は含まず
                                - 期間とは、どのくらいまで考慮するか
                            - **R(rate)**：減衰率
            - **飽和モデル**
                - 飽和（収穫逓減）を表現する数理モデルは数存在する。
                - **シンプルな効果モデル**
                    - 指数関数$1-\exp(-\alpha x)$
                    - ハイパーパラメータ：$\alpha$
            - **学習器**
                - **(仮)線形回帰モデル**
        - [**変換器 : 【2】**](#AdStock_logi)
            - **効果のピークが、広告を打った時に限らないモデル**
            - **ラグ(キャリーオーバー)効果モデル**
                - **数理モデル**
                - **効果モデル**
                    - $x_{t}^{*}=\frac{\displaystyle\sum_{l=0}^{L-1}w_{t-l}・x_{t-l}}{\displaystyle\sum_{l=0}^{L-1}w_{t-1}}, w_{t-l}=R^{(l-P)^{2}}$
                    - $x_{t}$は$t$期の広告などの投入量で、$x_{t}^{*}$は$t$期とそれ以前までの広告などの効果の累積(残存・ラグ効果を加算蓄積)
                    - **ハイパーパラメータ**
                        - **L(length)**：効果の続く期間　※当期含む
                        - **P(peak)**：ピークの時期（広告などを打った日の場合は0、次期は1、など）
                        - **R(rate)**：減衰率
            - **飽和モデル**
                - **S字曲線を表現する関数：シグモイド関数、ロジスティック曲線、ゴンペルツ曲線**
                - **ロジスティック曲線でのモデル**
                    - $y=\frac{K}{1+be^{c(x-m)}}$
                        - **K**：上限パラメータ
                        - **b**：形状パラメータ
                        - **c**：形状パラメータ
                        - **m**：位置パラメータ

- [参照サイト２](https://www.salesanalytics.co.jp/datascience/datascience099/)
- [参照サイト３](https://www.salesanalytics.co.jp/datascience/datascience100/)
- [参照サイト４](https://www.salesanalytics.co.jp/datascience/datascience119/)
- [参照サイト５：主成分分析回帰(PCR)](https://www.salesanalytics.co.jp/datascience/datascience122/).
- [参照サイト６：部分的最小2乗回帰(PLS)](https://www.salesanalytics.co.jp/datascience/datascience136/)

In [ ]:
import numpy as np
import pandas as pd


# Model
from sklearn.linear_model import LinearRegression

# CV,SPLIT
from sklearn.model_selection import cross_val_score, TimeSeriesSplit

# plot
import matplotlib.pyplot as plt
plt.style.use('ggplot') #グラフスタイル
plt.rcParams['figure.figsize'] = [16, 9] # グラフサイズ


# Plot設定：指数表記なし
np.set_printoptions(precision=3,suppress=True)
pd.options.display.float_format = '{:.3f}'.format

In [ ]:
# DataLoad
url = 'https://www.salesanalytics.co.jp/4zdt'
df = pd.read_csv(url,
                 parse_dates=['Week'],
                 index_col='Week'
                )

# データの確認
print(df.info()) #変数の情報
print(df.head())

# 説明変数Xと目的変数yに分解
X = df.drop(columns=['Sales']) 
y = df['Sales']

# グラフ化
y.plot()
X.plot()

In [ ]:
df

- **モデル構築**

In [ ]:
linearr = LinearRegression() # DF

- どの程度の予測精度を持ったモデルかを見る為に、**時系列におけるCV**を行う。**これをBaselineとする**
- デフォルト５：CVの結果（決定係数$R^{2}$）の平均値
    - **⇒　全てのデータを使用してモデルを構築する**

In [ ]:
# CV精度検証
np.mean(cross_val_score(linearr, X, y, 
                        cv=TimeSeriesSplit())
       )

In [ ]:
# モデルの学習
linearr.fit(X, y)

# データフレーム化
weights = pd.Series(linearr.coef_, index=X.columns)


print('Intercept:\n', linearr.intercept_, sep='') # 切片
print('\n')
print('Coefficients:\n',weights, sep='') # 係数

- **売上貢献度のを計算(補正前)**

In [ ]:
# 貢献度（補正前）
unadj_contribution = X.mul(weights) #Xと係数を乗算

unadj_contribution = unadj_contribution.assign(Base=linearr.intercept_) #切片の追加
unadj_contribution.head() #確認

- **週ごとに各媒体の売上貢献度を合計**
    - **売上の予測値**と**元の売上の実測値**

In [ ]:
# 貢献度の合計（yの予測値）
y_pred = unadj_contribution.sum(axis=1)

print('予測値　\n', y_pred.head())
print('テスト　\n', y.head())

- **乖離をなくすために、補正係数（correction factor）を計算し、売上貢献度を補正**
- **補正係数**を計算する

In [ ]:
# 補正係数
correction_factor = y.div(y_pred, axis=0)
correction_factor.head() #確認

- **補正係数を使い、売上貢献度を補正**

In [ ]:
# 貢献度（補正後）
adj_contribution = (unadj_contribution
                    .mul(correction_factor, axis=0)
                   ) 

# 順番の変更
adj_contribution = adj_contribution[['Base', 'Web', 'Newspaper', 'TVCM']]

#確認
adj_contribution.head()

- **週×媒体別の売上貢献度が求められた**
    - 積み上げグラフで可視化

In [ ]:
# グラフ化
ax = (adj_contribution
      .plot.area(
          figsize=(16, 10),
          linewidth=0,
          ylabel='Sales',
          xlabel='Date')
     )


handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[::-1], labels[::-1])

- 媒体別に全ての週の売上貢献度を合計し、媒体別の売上貢献度（円と構成比％）
- **何がどれほど売上に貢献したのか**

In [ ]:
# 媒体別の貢献度の合計
contribution_sum = adj_contribution.sum(axis=0)
#集計結果
print('売上貢献度（円）:\n', 
      contribution_sum, 
      sep=''
     )
print()
print('売上貢献度（構成比）:\n', 
      contribution_sum/contribution_sum.sum(), 
      sep=''
     )
#グラフ化
contribution_sum.plot.pie(fontsize=24)

- **費用対効果**
    - **媒体別のコストを集計し、媒体別にROIを計算**

In [ ]:
# 各媒体のコストの合計
cost_sum = X.sum(axis=0)
cost_sum #確認

- 先程求めた売上貢献度を使い、**媒体別のROIを計算しグラフ化**

In [ ]:
# 各媒体のROIの計算
ROI = (contribution_sum.drop('Base', axis=0) - cost_sum)/cost_sum

#確認
print('ROI:\n', ROI, sep='')

# グラフ化
ROI.plot.bar(fontsize=24)

- **ROIは、値が大きいほど良く、最低限プラスの値である必要があります。** 
    - **Web広告以外はあまり良くない**ことが分かる
- **【注意】このモデルには、ある致命的な欠陥があります。アドストック（Ad Stock）を考慮していない**
    - **アドストック（Ad Stock）を考慮**したモデルにした方がいい
        - **アドストック（Ad Stock）を考慮**するとは、**残存効果を考慮する**ということ
            - **ある日の広告宣伝活動が、その日だけに効果があるのではなく、次の日以降もその効果が続くということ**です。キャリーオーバー（CarryOver）効果と表現されることもある

***
<a id="AdStock_defo"></a>
# <p>**AdStockモデル構築 : Def**</p>

In [ ]:
import numpy as np
import pandas as pd


from scipy.signal import convolve2d


from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score, TimeSeriesSplit
from sklearn import set_config

# model
from sklearn.linear_model import LinearRegression

# Optuna
from optuna.integration import OptunaSearchCV
from optuna.distributions import UniformDistribution, IntUniformDistribution, FloatDistribution, IntDistribution, CategoricalDistribution

# Plot
import matplotlib.pyplot as plt
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = [16, 9]

#plot: 設定: 指数表記
np.set_printoptions(precision=3,suppress=True)
pd.options.display.float_format = '{:.3f}'.format

In [ ]:
# DataLoad
url = 'https://www.salesanalytics.co.jp/4zdt'
df = pd.read_csv(url,
                 parse_dates=['Week'],
                 index_col='Week'
                )

# 
print(df.info())
print(df.head()) 


# 説明変数Xと目的変数yに分解
X = df.drop(columns=['Sales'])
y = df['Sales']


# グラフ化
y.plot()
X.plot()

- **Baseline**
- **アドストックを考慮しない線形回帰モデル**を構築する
- **予測精度の確認 ： 時系列のCV**を実施

In [ ]:
# 線形回帰モデルのインスタンス
lr = LinearRegression()

In [ ]:
# クロスバリデーションで精度検証（R2）
np.mean(cross_val_score(lr, 
                        X, y, 
                        cv=TimeSeriesSplit()
                       )
       )

In [ ]:
# 全データで精度検証（R2）
lr.fit(X, y)
lr.score(X, y)

### **アドストック関数**
- **２つのモデル(変換器)の関数**を定義しAdStockを表現
    - **飽和モデル**
        - 指数関数$1-\exp(-\alpha x)$
        - ハイパーパラメータ : $\alpha$
    - **ラグ効果モデル(CaryOverModel)**
        - ハイパーパラメータ : **減退率$(rate)$** **期間$(length)$**
- [**参照サイト**](https://www.salesanalytics.co.jp/datascience/datascience098/)

In [ ]:
# 飽和モデル : 指数関数 1 - exp(-αx)
def Saturation(X, a):
    return 1 - np.exp(-a * X)

In [ ]:
# ラグ効果モデル
def Carryover(X: np.ndarray, rate, length):
    filter = (rate ** np.arange(length + 1)).reshape(-1, 1) 
    convolution = convolve2d(X, filter)
    if length > 0 : convolution = convolution[: -length]
    return convolution

- **飽和モデル**

In [ ]:
exp_dat = pd.DataFrame(range(500))
exp_sat = pd.DataFrame(index=exp_dat.index)

exp_sat['a=0.01'] = Saturation(exp_dat,0.01) 
exp_sat['a=0.02'] = Saturation(exp_dat,0.02) 
exp_sat['a=0.1'] = Saturation(exp_dat,0.1) 

print(exp_sat)

exp_sat.plot()

- **残差効果モデル**
- 最初100でその後0であるシンプルなケースの、この関数の実行例

In [ ]:
# Sample
exp_dat = pd.DataFrame([100,0,0,0,0,0,0,0,0,0]) 

# 残差
exp_co = Carryover(exp_dat, 0.5,2) 

print(exp_co) #数値出力
plt.bar(exp_dat.index,exp_co[:,0]) #グラフ
plt.title('rate=0.5, length=2')

- 先程の例よりも複雑なケースで、この関数の実行例
- サンプルデータ作成

In [ ]:
# サンプルデータ
exp_dat = pd.DataFrame([10,500,10,500])  #入力データ
print(exp_dat) #数値出力
plt.bar(exp_dat.index,exp_dat.values[:,0]) #グラフ

- サンプルデータに対し、この関数の実行

In [ ]:
exp_co = Carryover(exp_dat, 0.5,2)

print(exp_co) #数値出力
plt.bar(exp_dat.index,exp_co[:,0])

- 飽和モデルや、ラグ効果モデルを単体で使用するのではない
    - ⇒ 変数(入力) ⇒ ラグモデル ⇒ ラグモデル(出力) ⇒ 飽和モデル ⇒ 飽和モデル(出力) ⇒ モデル
    - 入力データ→飽和モデル→キャリーオーバー効果モデル→学習器のインプット

In [ ]:
# 「入力→ラグ効果モデル→飽和モデル→出力」の例
exp_sat_co = Saturation(Carryover(exp_dat, 0.5,2),0.01)

print(exp_sat_co) #数値出力
plt.bar(exp_dat.index,exp_sat_co[:,0]) #グラフ

### **アドストックを考慮した線形回帰モデル**
- **ハイパーパラメータ**の設定

In [ ]:
# TVCMのハイパーパラメータの設定
TVCM_carryover_rate = 0.5
TVCM_carryover_length = 4
TVCM_saturation_a = 0.000002

# Newspaperのハイパーパラメータの設定
Newspaper_carryover_rate = 0.5
Newspaper_carryover_length = 2
Newspaper_saturation_a = 0.000002

# Webのハイパーパラメータの設定
Web_carryover_rate = 0.5
Web_carryover_length = 0
Web_saturation_a = 0.000002

- **どのようなキャリーオーバー効果なのか**

In [ ]:
# キャリーオーバー効果モデルの出力例
## Sample Data
exp_dat = pd.DataFrame([100,0,0,0,0,0,0,0,0,0]) 

## ------------------------------------------
## キャリーオーバー効果
### TVCM
exp_co_TVCM= Carryover(
    exp_dat, 
    TVCM_carryover_rate,
    TVCM_carryover_length
) 
### Newspaper
exp_co_Newspaper = Carryover(
    exp_dat,   
    Newspaper_carryover_rate,
    Newspaper_carryover_length
) 
### Web
exp_co_Web = Carryover(
    exp_dat,    
    Web_carryover_rate,
    Web_carryover_length
) 
## -----------------------------------------

## グラフ
fig, axes = plt.subplots(nrows=1, ncols=3, sharex=False)
### TVCM
axes[0].bar(exp_dat.index,
            exp_co_TVCM[:,0]
           ) 
axes[0].set_title('TVCM')
### Newspaper
axes[1].bar(exp_dat.index,
            exp_co_Newspaper[:,0]
           ) 
axes[1].set_title('Newspaper')
### Web
axes[2].bar(exp_dat.index,
            exp_co_Web[:,0]
           ) 
axes[2].set_title('Web')

- 説明変数X（広告などのコスト）のデータに対し、**ラグ効果モデル**と**飽和モデル**を使い、**学習器（線形回帰モデル）のインプット**を作る

In [ ]:
# TVの値の変換
X_TVCM = Saturation(Carryover(X[['TVCM']],
                            TVCM_carryover_rate,
                            TVCM_carryover_length),
                  TVCM_saturation_a)
# Newspaperの値の変換
X_Newspaper = Saturation(Carryover(X[['Newspaper']], 
                               Newspaper_carryover_rate,
                               Newspaper_carryover_length),
                     Newspaper_saturation_a)
# Webの値の変換
X_Web = Saturation(Carryover(X[['Web']], 
                                 Web_carryover_rate,
                                 Web_carryover_length),
                       Web_saturation_a)
# 変換した値の結合（DataFrame型へ）
X_trans = pd.DataFrame(np.concatenate([X_TVCM, 
                                       X_Newspaper,
                                       X_Web], 
                                      1))

X_trans.columns = ['TVCM','Newspaper','Web']
X_trans #確認

- この説明変数Xを変換し作ったデータを使い、**線形回帰モデルを作り精度検証**
- 時系列のCV（クロスバリデーション）を実施

In [ ]:
# クロスバリデーションで精度検証（R2）
np.mean(cross_val_score(lr, 
                        X_trans, y, 
                        cv=TimeSeriesSplit()
                       )
       )

In [ ]:
# 全データで精度検証（R2）
lr.fit(X_trans, y)
lr.score(X_trans, y)

### **アドストックを考慮した線形回帰モデル（Optunaでハイパーパラメータチューニング）**
- **変換器と学習器を繋いだパイプライン**を作り、**Optunaでハイパーパラメータ探索**を実施し**最適なモデル**を作る

## **パイプライン構築**
- **飽和モデル**と**ラグ効果モデル**を、**パイプラインで利用できる変換器**

In [ ]:
# Pipeline用変換器（飽和モデル）
class ExponentialSaturation(BaseEstimator, TransformerMixin):
    
    # 初期化
    def __init__(self, a=1.0):
        self.a = a #ハイパーパラメータ
    
    # 学習
    def fit(self, X, y=None):
        return self
    
    # 処理（入力→出力）
    def transform(self, X):
        return Saturation(X,self.a)
    

# Pipeline用変換器（ラグ効果モデル）
class ExponentialCarryover(BaseEstimator, TransformerMixin):
    
    # 初期化
    def __init__(self, rate=0.5, length=1):
        self.rate = rate #ハイパーパラメータ
        self.length = length #ハイパーパラメータ
        
    # 学習        
    def fit(self, X, y=None):
        return self
    
    # 処理（入力→出力）
    def transform(self, X: np.ndarray):
        return Carryover(X, self.rate, self.length)

- **この変換器を使い、パイプラインを構築**

In [ ]:
# Pipelineの構築

## 説明変数の変換部分（adstock）の定義
adstock = ColumnTransformer(
    [
     ('TVCM_pipe', Pipeline([
                           ('carryover', ExponentialCarryover()),
                           ('saturation', ExponentialSaturation())
     ]), ['TVCM']),
     ('Newspaper_pipe', Pipeline([
                           ('carryover', ExponentialCarryover()),
                           ('saturation', ExponentialSaturation())
     ]), ['Newspaper']),
     ('Web_pipe', Pipeline([
                           ('carryover', ExponentialCarryover()),
                           ('saturation', ExponentialSaturation())
     ]), ['Web']),
    ],
    remainder='passthrough'
)

## 説明変数の変換（adstock）→線形回帰モデル（regression）
MMM_pipe = Pipeline([
    ('adstock', adstock),
    ('regression', LinearRegression())
])

## パイプラインの確認
set_config(display='diagram')   
MMM_pipe

### **パイパーパラメータ探索の実施**
- **パイプラインを構築**したので、**パイパーパラメータを探索**

In [ ]:
# 探索するハイパーパラメータの設定
params = {
        'adstock__TVCM_pipe__carryover__rate': FloatDistribution(0, 1),
        'adstock__TVCM_pipe__carryover__length': IntDistribution(0, 6),
        'adstock__TVCM_pipe__saturation__a': FloatDistribution(0, 0.01),
        'adstock__Newspaper_pipe__carryover__rate': FloatDistribution(0, 1),
        'adstock__Newspaper_pipe__carryover__length': IntDistribution(0, 6),
        'adstock__Newspaper_pipe__saturation__a': FloatDistribution(0, 0.01),
        'adstock__Web_pipe__carryover__rate': FloatDistribution(0, 1),
        'adstock__Web_pipe__carryover__length': IntDistribution(0, 6),
        'adstock__Web_pipe__saturation__a': FloatDistribution(0, 0.01),
}
# ハイパーパラメータ探索の設定
optuna_search = OptunaSearchCV(
    estimator=MMM_pipe,
    param_distributions=params,
    n_trials=1000,
    cv=TimeSeriesSplit(),
    random_state=0
)
# 探索実施
optuna_search.fit(X, y)

In [ ]:
# 探索結果
optuna_search.best_params_

- **探索し得られたハイパーパラメータ**で、**どのようなラグ効果になるのか**を確認する

In [ ]:
# ラグ効果モデルの出力例
## SampleData
exp_dat = pd.DataFrame([100,0,0,0,0,0,0,0,0,0])

## ハイパーパラメータ設定 : BestParams
### TVCM
TVCM_carryover_rate = optuna_search.best_params_['adstock__TVCM_pipe__carryover__rate']
TVCM_carryover_length = optuna_search.best_params_['adstock__TVCM_pipe__carryover__length']
### Newspaper
Newspaper_carryover_rate = optuna_search.best_params_['adstock__Newspaper_pipe__carryover__rate']
Newspaper_carryover_length = optuna_search.best_params_['adstock__Newspaper_pipe__carryover__length']
### Web
Web_carryover_rate = optuna_search.best_params_['adstock__Web_pipe__carryover__rate']
Web_carryover_length = optuna_search.best_params_['adstock__Web_pipe__carryover__length']


## キャリーオーバー効果
### TVCM
exp_co_TVCM= Carryover(
    exp_dat, 
    TVCM_carryover_rate,
    TVCM_carryover_length
) 
### Newspaper
exp_co_Newspaper = Carryover(
    exp_dat,   
    Newspaper_carryover_rate,
    Newspaper_carryover_length
) 
### Web
exp_co_Web = Carryover(
    exp_dat,    
    Web_carryover_rate,
    Web_carryover_length
) 


## Plot
fig, axes = plt.subplots(nrows=1, ncols=3, sharex=False)

### TVCM
axes[0].bar(exp_dat.index,
            exp_co_TVCM[:,0]
           ) 
axes[0].set_title('TVCM(y:logarithmic scale)')
axes[0].set_yscale('log')
axes[0].set_ylim(0, 100)

### Newspaper
axes[1].bar(exp_dat.index,
            exp_co_Newspaper[:,0]
           ) 
axes[1].set_title('Newspaper')

### Web
axes[2].bar(exp_dat.index,
            exp_co_Web[:,0]
           ) 
axes[2].set_title('Web')

### **最適なハイパーパラメータでパイプラインを学習**

In [ ]:
# パイプラインのインスタンス
MMM_pipe_best = MMM_pipe.set_params(**optuna_search.best_params_)

In [ ]:
# CVで精度検証
np.mean(cross_val_score(MMM_pipe_best, 
                        X, y, 
                        cv=TimeSeriesSplit()
                       )
       )

In [ ]:
# 全データで精度検証（R2）
MMM_pipe_best.fit(X, y)
MMM_pipe_best.score(X, y)

- どのような線形回帰モデルなのか（切片と回帰係数）を見る

In [ ]:
# 線形回帰モデルの切片と回帰係数
intercept = MMM_pipe_best.named_steps['regression'].intercept_ #切片
coef = MMM_pipe_best.named_steps['regression'].coef_ #回帰係数
# 回帰係数をデータフレーム化
weights = pd.Series(
    coef,
    index=X.columns
)
# 結果出力（切片と係数）
print('Intercept:\n', intercept, sep='')
print()
print('Coefficients:\n',weights, sep='')

### **売上貢献度の算出**
- 学習し構築した**パイプラインの変換器（adstock）**を抽出

In [ ]:
# Piplineの変換器（adstock）を抽出
adstock = MMM_pipe_best.named_steps['adstock']

- 説明変数を変換し、学習データのインプットデータを作る

In [ ]:
# 説明変数Xの変換
X_trans = pd.DataFrame(adstock.transform(X),
                       index=X.
                       index,columns=X.columns)

- 先程求めた線形回帰式を使い、売上貢献度（補正前）を計算

In [ ]:
# 貢献度（補正前）
unadj_contribution = X_trans.mul(weights) #Xと係数を乗算
unadj_contribution = unadj_contribution.assign(Base=intercept) #切片の追加

unadj_contribution.head() #確認

- 週ごとに各媒体の売上貢献度を合計すると、**売上の予測値**

In [ ]:
# 貢献度の合計（yの予測値）
y_pred = unadj_contribution.sum(axis=1)
y_pred.head() #確認

- 元の**売上の実測値**

In [ ]:
y.head() #確認

- **予測値と実測値が乖離**している。この乖離をなくすために、**補正係数**を計算し、**売上貢献度を補正**

In [ ]:
# 補正係数
correction_factor = y.div(y_pred, axis=0)
correction_factor.head() #確認

- この**補正係数**を使い、**売上貢献度を補正**

In [ ]:
# 貢献度（補正後）
adj_contribution = (unadj_contribution
                    .mul(correction_factor, axis=0)
                   ) 
# 順番の変更
adj_contribution = adj_contribution[['Base', 'Web', 'Newspaper', 'TVCM']]
#確認
adj_contribution.head()

- 週×媒体別の売上貢献度が求められたので、**積み上げグラフを作成**

In [ ]:
# グラフ化
ax = (adj_contribution
      .plot.area(
          figsize=(16, 10),
          linewidth=0,
          ylabel='Sales',
          xlabel='Date')
     )
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[::-1], labels[::-1])

- **媒体別に全ての週の売上貢献度を合計**し、**媒体別の売上貢献度（円と構成比％）**と、そのグラフを作り、**何がどれほど売上に貢献したのか**を見る

In [ ]:
# 媒体別の貢献度の合計
contribution_sum = adj_contribution.sum(axis=0)
#集計結果
print('売上貢献度（円）:\n', 
      contribution_sum, 
      sep=''
     )
print()
print('売上貢献度（構成比）:\n', 
      contribution_sum/contribution_sum.sum(), 
      sep=''
     )
#グラフ化
contribution_sum.plot.pie(fontsize=24)

- **費用対効果**を見てみます。今回は**媒体別にROIを計算**
- **媒体別のコスト**を集計

In [ ]:
# 各媒体のコストの合計
cost_sum = X.sum(axis=0)
cost_sum #確認

- 先程求めた売上貢献度を使い、**媒体別のROIを計算しグラフ化**

In [ ]:
# 各媒体のROIの計算
ROI = (contribution_sum.drop('Base', axis=0) - cost_sum)/cost_sum

#確認
print('ROI:\n', ROI, sep='')
# グラフ化
ROI.plot.bar(fontsize=24)

- ROIは、値が大きいほど良く、最低限プラスの値である必要があります。少なくとも0以上である必要がある
- このモデルは、**アドストック（Ad Stock）を考慮**していますが、**残差効果モデル**の作り方から分かりますが、**最初に効果のピークが来て徐々に効果が減衰するモデル**
- 現実はそうではなく、**効果のピークが最初に来ることもあれば、数日後や数週間後にピークが来ることもあります。** そのようなモデルの方がいい

- 今回は最適なハイパーパラメータを1つ探索するということをしましたが、別のハイパーパラメータの組み合わせでも同じ程度のレベルの予測精度になる可能性もあります。
- **ハイパーパラメータを1つ**とするのではなく、ある分布の代表値として見なす考え方もあります。そのためには、**ベイズモデルでMMMを構築する**必要がある

***
<a id="AdStock_logi"></a>
# <p>**AdStock :飽和モデル(ロジスティック関数)**</p>

### **飽和モデル**
- **ロジスティック曲線**
    - $y=\frac{K}{1+be^{c(x-m)}}$
        - **x**：データ
        - **K**：上限パラメータ
        - **b**：形状パラメータ
        - **c**：形状パラメータ
        - **m**：位置パラメータ
        - **e**：exp

In [ ]:
# 飽和モデル（関数）の定義
def Saturation(X,K,b,c,m):
    return K/(1+b*np.exp(-c*(X-m)))

In [ ]:
# 飽和モデル（関数）の例
exp_dat = pd.DataFrame(range(100)) #入力データ
exp_sat = pd.DataFrame(index=exp_dat.index)
exp_sat['K=1 b=1 c=1 m=50'] = Saturation(exp_dat,1,1,1,50) 
exp_sat['K=1 b=1 c=0.1 m=50'] = Saturation(exp_dat,1,1,0.1,50) 
exp_sat['K=1 b=2 c=0.1 m=50'] = Saturation(exp_dat,1,2,0.1,50) 
exp_sat['K=1 b=1 c=0.01 m=50'] = Saturation(exp_dat,1,1,0.01,50) 
exp_sat.plot() #グラフ

### **残差効果モデル**
- **効果のピークが広告などを打った時に限らない**モデル
- $x_{t}$期の広告などの投入量で、$x_{t}^{*}$は$t$期とそれ以前までの広告などの効果の蓄積(残差効果を足したもの)
- **ハイパーパラメータ**
    - **L(length)**：効果の続く期間（広告などを打った日も含む）
    - **P(peak)**：ピークの時期（広告などを打った日の場合は0、次期は1、など）
    - **R(rate)**：減衰率

In [ ]:
def Carryover(X: np.ndarray, length, peak, rate):
    X = np.append(np.zeros(length-1), X)
    
    Ws = np.zeros(length)
    
    for l in range(length):
        W = rate**((l-peak)**2)
        Ws[length-1-l] = W
    
    carryover_X = []
    
    for i in range(length-1, len(X)):
        X_array = X[i-length+1:i+1]
        Xi = sum(X_array * Ws)/sum(Ws)
        carryover_X.append(Xi)
        
    return np.array(carryover_X)

### **ラグ効果モデル**
- **効果のピークが広告などを打った時に限らない**モデル
- **ハイパーパラメータ**
    - **L(length)**：効果の続く期間（広告などを打った日も含む）
    - **P(peak)**：ピークの時期（広告などを打った日の場合は0、次期は1、など）
    - **R(rate)**：減衰率

In [ ]:
def Carryover(X: np.ndarray, length, peak, rate):
    X = np.append(np.zeros(length-1), X)
    
    Ws = np.zeros(length)
    
    for l in range(length):
        W = rate**((l-peak)**2)
        Ws[length-1-l] = W
    
    carryover_X = []
    
    for i in range(length-1, len(X)):
        X_array = X[i-length+1:i+1]
        Xi = sum(X_array * Ws)/sum(Ws)
        carryover_X.append(Xi)
        
    return np.array(carryover_X)

- **最初100**で**その後0**であるシンプルなケースの、この関数の**実行例**

In [ ]:
# キャリーオーバー効果モデル（関数）の例
## サンプルデータ
exp_dat = pd.DataFrame([100,0,0,0,0,0,0,0,0,0]) 
## キャリーオーバー効果
exp_co = Carryover(exp_dat, 4, 1, 0.5)
plt.bar(exp_dat.index,exp_co) #グラフ
plt.title('length=4 peak=1 rate=0.5')

### **アドストックを考慮した線形回帰モデル**

In [ ]:
# TVCMのハイパーパラメータの設定
TVCM_carryover_length = 5
TVCM_carryover_peak = 1
TVCM_carryover_rate = 0.5
TVCM_saturation_K = 1
TVCM_saturation_b = 1
TVCM_saturation_c = 0.1
TVCM_saturation_m = np.mean(X.TVCM)

# Newspaperのハイパーパラメータの設定
Newspaper_carryover_length = 3
Newspaper_carryover_peak = 0
Newspaper_carryover_rate = 0.5
Newspaper_saturation_K = 1
Newspaper_saturation_b = 1
Newspaper_saturation_c = 0.1
Newspaper_saturation_m = np.mean(X.Newspaper)

# Webのハイパーパラメータの設定
Web_carryover_length = 1
Web_carryover_peak = 0
Web_carryover_rate = 0.5
Web_saturation_K = 1
Web_saturation_b = 1
Web_saturation_c = 0.1
Web_saturation_m = np.mean(X.Web)

- **どのようなキャリーオーバー効果なのか**

In [ ]:
# キャリーオーバー効果モデルの出力例
## サンプルデータ
exp_dat = pd.DataFrame([100,0,0,0,0,0,0,0,0,0]) 
## キャリーオーバー効果
### TVCM
exp_co_TVCM= Carryover(
    exp_dat, 
    TVCM_carryover_length,
    TVCM_carryover_peak,
    TVCM_carryover_rate
) 
### Newspaper
exp_co_Newspaper = Carryover(
    exp_dat,   
    Newspaper_carryover_length,
    Newspaper_carryover_peak,
    Newspaper_carryover_rate
) 
### Web
exp_co_Web = Carryover(
    exp_dat,    
    Web_carryover_length,
    Web_carryover_peak,
    Web_carryover_rate
) 
## グラフ
fig, axes = plt.subplots(nrows=1, ncols=3, sharex=False)
### TVCM
axes[0].bar(exp_dat.index,
            exp_co_TVCM
           ) 
axes[0].set_title('TVCM')
### Newspaper
axes[1].bar(exp_dat.index,
            exp_co_Newspaper
           ) 
axes[1].set_title('Newspaper')
### Web
axes[2].bar(exp_dat.index,
            exp_co_Web
           ) 
axes[2].set_title('Web')

- 説明変数X（広告などのコスト）のデータに対し、**ラグ効果モデルと飽和モデル**を使い、**学習器（線形回帰モデル）のインプット**

In [ ]:
# TVの値の変換
X_TVCM = Saturation(Carryover(X[['TVCM']],
                              TVCM_carryover_length,
                              TVCM_carryover_peak,
                              TVCM_carryover_rate).reshape(-1,1),
                    TVCM_saturation_K,
                    TVCM_saturation_b,
                    TVCM_saturation_c,
                    TVCM_saturation_m
                   )
# Newspaperの値の変換
X_Newspaper = Saturation(Carryover(X[['Newspaper']], 
                              Newspaper_carryover_length,
                              Newspaper_carryover_peak,
                              Newspaper_carryover_rate).reshape(-1,1),
                    Newspaper_saturation_K,
                    Newspaper_saturation_b,
                    Newspaper_saturation_c,
                    Newspaper_saturation_m
                   )
# Webの値の変換
X_Web = Saturation(Carryover(X[['Web']], 
                              Web_carryover_length,
                              Web_carryover_peak,
                              Web_carryover_rate).reshape(-1,1),
                    Web_saturation_K,
                    Web_saturation_b,
                    Web_saturation_c,
                    Web_saturation_m
                   )
# 変換した値の結合（DataFrame型へ）
X_trans = pd.DataFrame(np.concatenate([X_TVCM, 
                                       X_Newspaper,
                                       X_Web], 
                                      1))
X_trans.columns = ['TVCM','Newspaper','Web']
X_trans #確認

In [ ]:
# クロスバリデーションで精度検証（R2）
# CV
np.mean(cross_val_score(lr, 
                        X_trans, y, 
                        cv=TimeSeriesSplit()
                       )
       )

In [ ]:
# 全データで精度検証（R2）
lr.fit(X_trans, y)
lr.score(X_trans, y)

### **AdStock - Optuna - Hyperparameters**
- 変換器と学習器を繋いだパイプライン
- Optunaでハイパーパラメータ探索を実施し最適なモデル
- **ハイパーパラメータ**
    - ラグ効果モデル
        - L(length)：効果の続く期間（広告などを打った日も含む）
        - P(peak)：ピークの時期（広告などを打った日の場合は0、次期は1、など）
        - R(rate)：減衰率
    - 飽和モデル
        - K：上限パラメータ
        - b：形状パラメータ
        - c：形状パラメータ
        - m：位置パラメータ
- **パイプライン構築**
    - 飽和モデルとラグ効果モデルを、パイプラインで利用できる変換器

In [ ]:
# Pipeline用変換器（飽和モデル）
class ExponentialSaturation(BaseEstimator, TransformerMixin):
    
    # 初期化
    def __init__(self, K=1.0, b=1.0, c=0.1, m=100.0):
        self.K = K
        self.b = b
        self.c = c
        self.m = m
    
    # 学習
    def fit(self, X, y=None):
        return self
    
    # 処理（入力→出力）
    def transform(self, X):
        return Saturation(X,self.K, self.b, self.c, self.m).reshape(-1,1)
    
# Pipeline用変換器（キャリーオーバー効果モデル）
class ExponentialCarryover(BaseEstimator, TransformerMixin):
    # 初期化
    def __init__(self, length=4, peak=1, rate=0.5):
        self.length = length #ハイパーパラメータ
        self.peak = peak #ハイパーパラメータ
        self.rate = rate #ハイパーパラメータ
        
    # 学習        
    def fit(self, X, y=None):
        return self
    
    # 処理（入力→出力）
    def transform(self, X: np.ndarray):
        return Carryover(X, self.length, self.peak, self.rate)

- **パイプライン構築**

In [ ]:
# Pipelineの構築
## 説明変数の変換部分（adstock）の定義
adstock = ColumnTransformer(
    [
     ('TVCM_pipe', Pipeline([
                           ('carryover', ExponentialCarryover()),
                           ('saturation', ExponentialSaturation())
     ]), ['TVCM']),
     ('Newspaper_pipe', Pipeline([
                           ('carryover', ExponentialCarryover()),
                           ('saturation', ExponentialSaturation())
     ]), ['Newspaper']),
     ('Web_pipe', Pipeline([
                           ('carryover', ExponentialCarryover()),
                           ('saturation', ExponentialSaturation())
     ]), ['Web']),
    ],
    remainder='passthrough'
)


## 説明変数の変換（adstock）→線形回帰モデル（regression）
MMM_pipe = Pipeline([
    ('adstock', adstock),
    ('regression', LinearRegression())
])


## パイプラインの確認
set_config(display='diagram')   
MMM_pipe

- **パイプラインを構築**したので、**パイパーパラメータを探索**

In [ ]:
# ハイパーパラメータの探索の設定
params = {
    'adstock__TVCM_pipe__carryover__length': IntUniformDistribution(1, 7),
    'adstock__TVCM_pipe__carryover__peak': IntUniformDistribution(0, 2),
    'adstock__TVCM_pipe__carryover__rate': UniformDistribution(0, 1),
    'adstock__TVCM_pipe__saturation__K': UniformDistribution(np.min(X.TVCM), np.max(X.TVCM)),
    'adstock__TVCM_pipe__saturation__b': UniformDistribution(0, 10),
    'adstock__TVCM_pipe__saturation__c': UniformDistribution(0, 1),
    'adstock__TVCM_pipe__saturation__m': UniformDistribution(np.min(X.TVCM), np.max(X.TVCM)),
    'adstock__Newspaper_pipe__carryover__length': IntUniformDistribution(1, 7),
    'adstock__Newspaper_pipe__carryover__peak': IntUniformDistribution(0, 2),
    'adstock__Newspaper_pipe__carryover__rate': UniformDistribution(0, 1),
    'adstock__Newspaper_pipe__saturation__K': UniformDistribution(np.min(X.Newspaper), np.max(X.Newspaper)),
    'adstock__Newspaper_pipe__saturation__b': UniformDistribution(0, 10),
    'adstock__Newspaper_pipe__saturation__c': UniformDistribution(0, 1),
    'adstock__Newspaper_pipe__saturation__m': UniformDistribution(np.min(X.Newspaper), np.max(X.Newspaper)),
    'adstock__Web_pipe__carryover__length': IntUniformDistribution(1, 7),
    'adstock__Web_pipe__carryover__peak': IntUniformDistribution(0, 2),
    'adstock__Web_pipe__carryover__rate': UniformDistribution(0, 1),
    'adstock__Web_pipe__saturation__K': UniformDistribution(np.min(X.Web), np.max(X.Web)),
    'adstock__Web_pipe__saturation__b': UniformDistribution(0, 10),
    'adstock__Web_pipe__saturation__c': UniformDistribution(0, 1),
    'adstock__Web_pipe__saturation__m': UniformDistribution(np.min(X.Web), np.max(X.Web)),
}

# ハイパーパラメータ探索の設定
optuna_search = OptunaSearchCV(
    estimator=MMM_pipe,
    param_distributions=params,
    n_trials=1000,
    cv=TimeSeriesSplit(),
    error_score='raise',
    random_state=0
)

# 探索実施
optuna_search.fit(X, y)

In [ ]:
# 探索結果
optuna_search.best_params_

- **探索し得られたハイパーパラメータ**で、どのような**ラグ効果**になるのか

In [ ]:
# キャリーオーバー効果モデルの出力例
## サンプルデータ
exp_dat = pd.DataFrame([100,0,0,0,0,0,0,0,0,0]) 


## ハイパーパラメータ設定
### TVCM
TVCM_carryover_length = optuna_search.best_params_['adstock__TVCM_pipe__carryover__length']
TVCM_carryover_peak = optuna_search.best_params_['adstock__TVCM_pipe__carryover__peak']
TVCM_carryover_rate = optuna_search.best_params_['adstock__TVCM_pipe__carryover__rate']
### Newspaper
Newspaper_carryover_length = optuna_search.best_params_['adstock__Newspaper_pipe__carryover__length']
Newspaper_carryover_peak = optuna_search.best_params_['adstock__Newspaper_pipe__carryover__peak']
Newspaper_carryover_rate = optuna_search.best_params_['adstock__Newspaper_pipe__carryover__rate']
### Web
Web_carryover_length = optuna_search.best_params_['adstock__Web_pipe__carryover__length']
Web_carryover_peak = optuna_search.best_params_['adstock__Web_pipe__carryover__peak']
Web_carryover_rate = optuna_search.best_params_['adstock__Web_pipe__carryover__rate']


## キャリーオーバー効果
### TVCM
exp_co_TVCM= Carryover(
    exp_dat, 
    TVCM_carryover_length,
    TVCM_carryover_peak,
    TVCM_carryover_rate,
) 
### Newspaper
exp_co_Newspaper = Carryover(
    exp_dat,   
    Newspaper_carryover_length,
    Newspaper_carryover_peak,
    Newspaper_carryover_rate,
) 
### Web
exp_co_Web = Carryover(
    exp_dat,    
    Web_carryover_length,
    Web_carryover_peak,
    Web_carryover_rate,
) 


## グラフ
fig, axes = plt.subplots(nrows=1, ncols=3, sharex=False)
### TVCM
axes[0].bar(exp_dat.index,
            exp_co_TVCM
           ) 
axes[0].set_title('TVCM(y:logarithmic scale)')
axes[0].set_yscale('log')
axes[0].set_ylim(0, 100)
### Newspaper
axes[1].bar(exp_dat.index,
            exp_co_Newspaper
           ) 
axes[1].set_title('Newspaper')
### Web
axes[2].bar(exp_dat.index,
            exp_co_Web
           ) 
axes[2].set_title('Web')

- 最適なハイパーパラメータでパイプラインを学習
- パイプラインのインスタンスを作成

In [ ]:
# パイプラインのインスタンス
MMM_pipe_best = MMM_pipe.set_params(**optuna_search.best_params_)

- **CV**

In [ ]:
# クロスバリデーションで精度検証（R2）
np.mean(cross_val_score(MMM_pipe_best, 
                        X, y, 
                        cv=TimeSeriesSplit()
                       )
       )

In [ ]:
# 全データで精度検証（R2）
MMM_pipe_best.fit(X, y)
MMM_pipe_best.score(X, y)

- 線形回帰モデルなのか（切片と回帰係数）

In [ ]:
# 線形回帰モデルの切片と回帰係数
intercept = MMM_pipe_best.named_steps['regression'].intercept_ #切片
coef = MMM_pipe_best.named_steps['regression'].coef_ #回帰係数
# 回帰係数をデータフレーム化
weights = pd.Series(
    coef,
    index=X.columns
)

# 結果出力（切片と係数）
print('Intercept:\n', intercept, sep='')
print()
print('Coefficients:\n',weights, sep='')

### **売上貢献度の算出**
- 学習し構築した**パイプラインの変換器（adstock）**を抽出

In [ ]:
# Piplineの変換器（adstock）を抽出
adstock = MMM_pipe_best.named_steps['adstock']

- 説明変数を変換し、**学習データのインプットデータ**を作る

In [ ]:
# 説明変数Xの変換
X_trans = pd.DataFrame(adstock.transform(X),
                       index=X.
                       index,columns=X.columns)

- 先程求めた線形回帰式を使い、**売上貢献度（補正前）を計算**

In [ ]:
# 貢献度（補正前）
unadj_contribution = X_trans.mul(weights) #Xと係数を乗算
unadj_contribution = unadj_contribution.assign(Base=intercept) #切片の追加
unadj_contribution.head() #確認

- 週ごとに各媒体の売上貢献度を合計すると、**売上の予測値**

In [ ]:
# 貢献度の合計（yの予測値）
y_pred = unadj_contribution.sum(axis=1)
y_pred.head() #確認

In [ ]:
y.head() #確認

- **予測値と実測値が乖離**していることが分かります。この乖離をなくすために、**補正係数（correction factor）**を計算し、**売上貢献度を補正**
- **補正係数を計算**

In [ ]:
# 補正係数
correction_factor = y.div(y_pred, axis=0)
correction_factor.head() #確認

- この**補正係数**を使い、**売上貢献度を補正**

In [ ]:
# 貢献度（補正後）
adj_contribution = (unadj_contribution
                    .mul(correction_factor, axis=0)
                   ) 
# 順番の変更
adj_contribution = adj_contribution[['Base', 'Web', 'Newspaper', 'TVCM']]
#確認
adj_contribution.head()

- **週×媒体別の売上貢献度**が求められたので、**積み上げグラフ作成**

In [ ]:
# グラフ化
ax = (adj_contribution
      .plot.area(
          figsize=(16, 10),
          linewidth=0,
          ylabel='Sales',
          xlabel='Date')
     )
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[::-1], labels[::-1])

- **媒体別に全ての週の売上貢献度を合計**し、**媒体別の売上貢献度（円と構成比％）**と、そのグラフを作り、何がどれほど売上に貢献したのかを見

In [ ]:
# 媒体別の貢献度の合計
contribution_sum = adj_contribution.sum(axis=0)
#集計結果
print('売上貢献度（円）:\n', 
      contribution_sum, 
      sep=''
     )
print()
print('売上貢献度（構成比）:\n', 
      contribution_sum/contribution_sum.sum(), 
      sep=''
     )
#グラフ化
contribution_sum.plot.pie(fontsize=24)

- **費用対効果**を見てみます。今回は**媒体別にROIを計算**します。

- 先ず、**媒体別のコスト**を集計

In [ ]:
# 各媒体のコストの合計
cost_sum = X.sum(axis=0)
cost_sum #確認

- 先程求めた売上貢献度を使い、**媒体別のROIを計算しグラフ化**

In [ ]:
# 各媒体のROIの計算
ROI = (contribution_sum.drop('Base', axis=0) - cost_sum)/cost_sum
#確認
print('ROI:\n', ROI, sep='')
# グラフ化
ROI.plot.bar(fontsize=24)

- **ROI**は、値が大きいほど良く、最低限プラスの値である必要があります。少なくとも0以上である必要があります。

***
<a id="Saturation"></a>
<a id="Sat_Converter"></a>
## <p>**飽和モデル・変換器クラス**</p>

- **指数型飽和モデル(exp_Saturation)**
- **変換器クラス**

In [ ]:
# 指数型飽和モデル（exp_Saturation）
def exp_Saturation(X,a):
    '''
    Hyperparameter : a
    '''
    return 1 - np.exp(-a*X)

In [ ]:
# 指数型飽和モデル（exp_Saturation）変換器
class pipe_exp_Saturation(BaseEstimator, TransformerMixin):
    
    # 初期化
    def __init__(self,a=1.0):
        self.a = a
    
    # 学習
    def fit(self, X, y=None):
        return self
    
    # 処理（入力→出力）
    def transform(self, X):
        X_ = exp_Saturation(X,
                            self.a
                           ).reshape(-1,1)
        return X_

- **ロジスティック型飽和モデル(logit_Saturatrion)**
- **変換器クラス**

In [ ]:
# ロジスティック型飽和モデル（logit_Saturation）
def logit_Saturation(X,K,b,c,m):
    '''
    Hyperparameters : k, b, c, m
    data : X
    '''
    return K/(1+b*np.exp(-c*(X-m)))

In [ ]:
# ロジスティック型飽和モデル（logit_Saturation）変換器
class pipe_logit_Saturation(BaseEstimator, TransformerMixin):
    
    # 初期化
    def __init__(self,K=1.0,m=100.0,b=1.0, c=0.1):
        self.K = K
        self.m = m
        self.b = b
        self.c = c
    
    # 学習
    def fit(self, X, y=None):
        return self
    
    # 処理（入力→出力）
    def transform(self, X):
        X_ = logit_Saturation(X,
                              self.K, 
                              self.b, 
                              self.c, 
                              self.m
                             ).reshape(-1,1)
        return X_

- **ゴンペルツ型飽和モデル(gom_Saturation)**
- **変換器**

In [ ]:
# ゴンペルツ型飽和モデル（gom_Saturation）
def gom_Saturation(X,K,b,c,m):
    '''
    Hyperparameters : k, b, c, m
    data : X
    '''
    return K*(b**np.exp(-c*(X-m)))

In [ ]:
# ゴンペルツ型飽和モデル（gom_Saturation）
class pipe_gom_Saturation(BaseEstimator, TransformerMixin):
    
    # 初期化
    def __init__(self,K=1.0,m=100.0,b=1.0,c=0.1):
        self.K = K
        self.m = m
        self.b = b
        self.c = c
    
    # 学習
    def fit(self, X, y=None):
        return self
    
    # 処理（入力→出力）
    def transform(self, X):
        X_ = gom_Saturation(X,
                            self.K, 
                            self.b, 
                            self.c, 
                            self.m
                           ).reshape(-1,1)
        return X_

***
<a id="Converter"></a>
<a id="lag_Converter"></a>
## <p>**ラグ効果モデル・変換器**</p>
- **定率減少型キャリーオーバー効果モデル　※ピークが広告などの投入時（simple_Carryover）**
- **変換器**

In [ ]:
# 定率減少型キャリーオーバー効果モデル　※ピークが広告などの投入時（simple_Carryover）
def simple_Carryover(X: np.ndarray, rate, length):
    
    filter = (
        rate ** np.arange(length + 1)
    ).reshape(-1, 1) 
    
    convolution = convolve2d(X, filter)
    
    if length > 0 : convolution = convolution[: -length]
        
    return convolution[:,0]

In [ ]:
# 定率減少型キャリーオーバー効果モデル　※ピークが広告など投入時（simple_Carryover）
class pipe_simple_Carryover(BaseEstimator, TransformerMixin):
    # 初期化
    def __init__(self,length=4,rate=0.5):
        self.length = length
        self.rate = rate
        
    # 学習        
    def fit(self, X, y=None):
        return self
    # 処理（入力→出力）
    def transform(self, X):
        X_ = simple_Carryover(X, 
                              self.rate, 
                              self.length
                             )
        return X_

- **ピーク可変型キャリーオーバー効果モデル（peak_Carryover）**
- **変換器**

In [ ]:
# ピーク可変型キャリーオーバー効果モデル（peak_Carryover）
def peak_Carryover(X: np.ndarray, length, peak, rate):
    X = np.append(np.zeros(length-1), X)
    
    Ws = np.zeros(length)
    
    for l in range(length):
        W = rate**((l-peak)**2)
        Ws[length-1-l] = W
    
    carryover_X = []
    
    for i in range(length-1, len(X)):
        X_array = X[i-length+1:i+1]
        Xi = sum(X_array * Ws)/sum(Ws)
        carryover_X.append(Xi)
        
    return np.array(carryover_X)

In [ ]:
# Pipeline用変換器（キャリーオーバー効果モデル）
class pipe_peak_Carryover(BaseEstimator, TransformerMixin):
    # 初期化
    def __init__(self,length=4,peak=1,rate=0.5):
        self.length = length
        self.peak = peak
        self.rate = rate
        
    # 学習        
    def fit(self, X, y=None):
        return self
    # 処理（入力→出力）
    def transform(self, X):
        X_ = peak_Carryover(X, 
                            self.length, 
                            self.peak, 
                            self.rate
                           ) 
        return X_

- **Optuna**

In [ ]:
# 目的関数の設定
def objective(trial):
    
    #TVCM
    
    ##TVCM Adstock func
    
    TVCM_Saturation_func = trial.suggest_categorical(
        "TVCM_Saturation_func",
        ["exp", "logit","gom"]
    )
    
    TVCM_Carryover_func = trial.suggest_categorical(
        "TVCM_Carryover_func",
        ["simple", "peak"]
    )
    
    ##TVCM_Saturation
    
    if TVCM_Saturation_func == 'exp':
        
        TVCM_a = trial.suggest_float(
            "TVCM_a", 
            0, 0.01
        )
        
        TVCM_Saturation = pipe_exp_Saturation(a=TVCM_a)
        
    elif TVCM_Saturation_func == 'logit':
        
        TVCM_K = trial.suggest_float(
            "TVCM_K", 
            np.min(X.TVCM), np.max(X.TVCM)*2
        )
        TVCM_m = trial.suggest_float(
            "TVCM_m",
            np.min(X.TVCM), np.max(X.TVCM)
        )
        TVCM_b = trial.suggest_float(
            "TVCM_b",
            0, 10
        )
        TVCM_c = trial.suggest_float(
            "TVCM_c",
            0, 1
        )
        
        TVCM_Saturation = pipe_logit_Saturation(
            K=TVCM_K,
            m=TVCM_m,
            b=TVCM_b,
            c=TVCM_c
        )
        
    else:
        
        TVCM_K = trial.suggest_float(
            "TVCM_K", 
            np.min(X.TVCM), 
            np.max(X.TVCM)*2
        )
        TVCM_m = trial.suggest_float(
            "TVCM_m", 
            np.min(X.TVCM), 
            np.max(X.TVCM)
        )
        TVCM_b = trial.suggest_float(
            "TVCM_b", 
            0, 1
        )
        TVCM_c = trial.suggest_float(
            "TVCM_c", 
            0, 1
        )     
        
        TVCM_Saturation = pipe_gom_Saturation(
            K=TVCM_K,
            m=TVCM_m,
            b=TVCM_b,
            c=TVCM_c
        )
        
    ##TVCM Carryover
    
    if TVCM_Carryover_func == 'simple':
        
        TVCM_rate = trial.suggest_float(
            "TVCM_rate", 
            0,1
        )
        TVCM_length = trial.suggest_int(
            "TVCM_length", 
            0,6
        )
        
        TVCM_Carryover = pipe_simple_Carryover(
            length=TVCM_length,
            rate=TVCM_rate
        )
        
    else:
        
        TVCM_rate = trial.suggest_float(
            "TVCM_rate", 
            0, 1
        )
        TVCM_length = trial.suggest_int(
            "TVCM_length",
            1,7
        )
        TVCM_peak = trial.suggest_int(
            "TVCM_peak",
            0,2
        )
        
        TVCM_Carryover = pipe_peak_Carryover(
            length=TVCM_length, 
            rate=TVCM_rate,
            peak=TVCM_peak
        )   
    #Newspaper
    
    ##Newspaper Adstock func
    
    Newspaper_Saturation_func = trial.suggest_categorical(
        "Newspaper_Saturation_func",
        ["exp", "logit","gom"]
    )
    
    Newspaper_Carryover_func = trial.suggest_categorical(
        "Newspaper_Carryover_func",
        ["simple", "peak"]
    )
    
    ##Newspaper_Saturation
    
    if Newspaper_Saturation_func == 'exp':
        
        Newspaper_a = trial.suggest_float(
            "Newspaper_a", 
            0, 0.01
        )
        
        Newspaper_Saturation = pipe_exp_Saturation(a=Newspaper_a)
        
    elif Newspaper_Saturation_func == 'logit':
        
        Newspaper_K = trial.suggest_float(
            "Newspaper_K",
            np.min(X.Newspaper), np.max(X.Newspaper)*2
        )
        Newspaper_m = trial.suggest_float(
            "Newspaper_m", 
            np.min(X.Newspaper), np.max(X.Newspaper)
        )
        Newspaper_b = trial.suggest_float(
            "Newspaper_b", 
            0, 10
        )
        Newspaper_c = trial.suggest_float(
            "Newspaper_c", 
            0, 1
        )
        
        Newspaper_Saturation = pipe_logit_Saturation(
            K=Newspaper_K,
            m=Newspaper_m,
            b=Newspaper_b,
            c=Newspaper_c
        )
        
    else:
        
        Newspaper_K = trial.suggest_float(
            "Newspaper_K", 
            np.min(X.Newspaper), np.max(X.Newspaper)*2
        )
        Newspaper_m = trial.suggest_float(
            "Newspaper_m", 
            np.min(X.Newspaper), np.max(X.Newspaper)
        )
        Newspaper_b = trial.suggest_float(
            "Newspaper_b", 
            0, 1
        )
        Newspaper_c = trial.suggest_float(
            "Newspaper_c", 
            0, 1
        )     
        
        Newspaper_Saturation = pipe_gom_Saturation(
            K=Newspaper_K,
            m=Newspaper_m,
            b=Newspaper_b,
            c=Newspaper_c
        )
        
    ##Newspaper Carryover
    
    if Newspaper_Carryover_func == 'simple':
        
        Newspaper_rate = trial.suggest_float(
            "Newspaper_rate", 
            0,1
        )
        Newspaper_length = trial.suggest_int(
            "Newspaper_length", 
            0,6
        )
        
        Newspaper_Carryover = pipe_simple_Carryover(
            length=Newspaper_length,
            rate=Newspaper_rate
        )
        
    else:
        
        Newspaper_rate = trial.suggest_float(
            "Newspaper_rate", 
            0, 1
        )
        Newspaper_length = trial.suggest_int(
            "Newspaper_length",
            1,7
        )
        Newspaper_peak = trial.suggest_int(
            "Newspaper_peak",
            0,2
        )
        
        Newspaper_Carryover = pipe_peak_Carryover(
            length=Newspaper_length,
            rate=Newspaper_rate,
            peak=Newspaper_peak
        )
            
    #Web
    
    ##Web Adstock func
    
    Web_Saturation_func = trial.suggest_categorical(
        "Web_Saturation_func",
        ["exp", "logit","gom"]
    )
    
    Web_Carryover_func = trial.suggest_categorical(
        "Web_Carryover_func",
        ["simple", "peak"]
    )
    
    ##Web_Saturation
    
    if Web_Saturation_func == 'exp':
        
        Web_a = trial.suggest_float(
            "Web_a", 
            0, 0.01
        )
        
        Web_Saturation = pipe_exp_Saturation(a=Web_a)
        
    elif Web_Saturation_func == 'logit':
        
        Web_K = trial.suggest_float(
            "Web_K", 
            np.min(X.Web), np.max(X.Web)*2
        )
        Web_m = trial.suggest_float(
            "Web_m", 
            np.min(X.Web), np.max(X.Web)
        )
        Web_b = trial.suggest_float(
            "Web_b", 
            0, 10
        )
        Web_c = trial.suggest_float(
            "Web_c", 
            0, 1
        )
        
        Web_Saturation = pipe_logit_Saturation(
            K=Web_K,
            m=Web_m,
            b=Web_b,
            c=Web_c
        )
        
    else:
        
        Web_K = trial.suggest_float(
            "Web_K", 
            np.min(X.Web), np.max(X.Web)*2
        )
        Web_m = trial.suggest_float(
            "Web_m", 
            np.min(X.Web), np.max(X.Web)
        )
        Web_b = trial.suggest_float(
            "Web_b", 
            0, 1
        )
        Web_c = trial.suggest_float(
            "Web_c", 
            0, 1
        )     
        
        Web_Saturation = pipe_gom_Saturation(
            K=Web_K,
            m=Web_m,
            b=Web_b,
            c=Web_c
        )
        
    ##Web Carryover
    
    if Web_Carryover_func == 'simple':
        
        Web_rate = trial.suggest_float(
            "Web_rate", 
            0,1
        )
        Web_length = trial.suggest_int(
            "Web_length", 
            0,6
        )
        
        Web_Carryover = pipe_simple_Carryover(
            length=Web_length,
            rate=Web_rate
        )
        
    else:
        
        Web_rate = trial.suggest_float(
            "Web_rate", 
            0, 1
        )
        Web_length = trial.suggest_int(
            "Web_length",
            1,7
        )
        Web_peak = trial.suggest_int(
            "Web_peak",
            0,2
        )
        
        Web_Carryover = pipe_peak_Carryover(
            length=Web_length,
            rate=Web_rate,
            peak=Web_peak
        )   
    
    # パイプライン化
    
    ## 変換器（Adstock）
    adstock = ColumnTransformer(
        [
         ('TVCM_pipe', Pipeline([
             ('TVCM_carryover', TVCM_Carryover),
             ('TVCM_saturation', TVCM_Saturation)
         ]), ['TVCM']),
         ('Newspaper_pipe', Pipeline([
             ('Newspaper_carryover', Newspaper_Carryover),
             ('Newspaper_saturation', Newspaper_Saturation)
         ]), ['Newspaper']),
         ('Web_pipe', Pipeline([
             ('Web_carryover', Web_Carryover),
             ('Web_saturation', Web_Saturation)
         ]), ['Web']),
        ],
        remainder='passthrough'
    )
    ## 説明変数の変換（adstock）→線形回帰モデル（regression）
    MMM_pipe = Pipeline([
        ('adstock', adstock),
        ('regression', LinearRegression())
    ])
        
    #CVによる評価
    score = cross_val_score(
        MMM_pipe,
        X,
        y,
        cv=TimeSeriesSplit()
    )
    accuracy = score.mean()
    return accuracy

- **目的関数に対し最適化を実行**することで、**最適な関数の組み合わせとそれぞれのハイパーパラメータを探索**

In [ ]:
# 目的関数の最適化を実行する
study = optuna.create_study(direction="maximize")
study.optimize(objective,
               n_trials=5000,
               show_progress_bar=True)

In [ ]:
# 探索結果
study.best_params

- **最適なハイパーパラメータでパイプラインを学習**
- **求めた最適なハイパーパラメータでモデル構築し精度検証**
- 具体的には、先ず**パイプラインの定義**をします。次に、そのパイプラインに最適なハイパーパラメータを設定します。そのパイプラインを使い、精度検証
### **パイプライン構築**
- パイプライン用の、**飽和モデル**と**キャリーオーバー効果の2つの変換器クラスを定義**

In [ ]:
# Pipeline用変換器
## 飽和モデル
class Saturation(BaseEstimator, TransformerMixin):
    
    # 初期化
    def __init__(self,func='exp',a=1.0,K=1.0,m=100.0,b=1.0, c=0.1):
        self.func = func
        self.a = a
        self.K = K
        self.m = m
        self.b = b
        self.c = c
    
    # 学習
    def fit(self, X, y=None):
        return self
    
    # 処理（入力→出力）
    def transform(self, X):
        if self.func == 'exp':
            X_ = exp_Saturation(X,
                                self.a
                               ).reshape(-1,1)
        elif self.func == "logit":
            X_ = logit_Saturation(X,
                                  self.K, 
                                  self.b,
                                  self.c, 
                                  self.m
                                 ).reshape(-1,1)
        else:
            X_ = gom_Saturation(X,
                                self.K, 
                                self.b, 
                                self.c, 
                                self.m
                               ).reshape(-1,1)
        return X_

In [ ]:
## ラグ効果モデル
class Carryover(BaseEstimator, TransformerMixin):
    # 初期化
    def __init__(self,func='simple',length=4, peak=1, rate=0.5):
        self.func = func
        self.length = length
        self.peak = peak
        self.rate = rate
        
    # 学習        
    def fit(self, X, y=None):
        return self
    # 処理（入力→出力）
    def transform(self, X):
        if self.func == 'peak':
            X_ = peak_Carryover(X, 
                                self.length, 
                                self.peak, 
                                self.rate)  
        else:
            X_ = simple_Carryover(X, 
                                  self.rate, 
                                  self.length)
            
        return X_

- **2つの変換器**で**アドストック（Ad Stock）**を表現します。これらの**変換器と学習器（線形回帰モデル）をつなげたパイプラインを構築**

In [ ]:
# Pipelineの設定
## 説明変数の変換部分（adstock）の定義
adstock = ColumnTransformer(
    [
     ('TVCM_pipe', Pipeline([
                           ('carryover', Carryover()),
                           ('saturation', Saturation())
     ]), ['TVCM']),
     ('Newspaper_pipe', Pipeline([
                           ('carryover', Carryover()),
                           ('saturation', Saturation())
     ]), ['Newspaper']),
     ('Web_pipe', Pipeline([
                           ('carryover', Carryover()),
                           ('saturation', Saturation())
     ]), ['Web']),
    ],
    remainder='passthrough'
)

## 説明変数の変換（adstock）→線形回帰モデル（regression）
MMM_pipe = Pipeline([
    ('adstock', adstock),
    ('regression', LinearRegression())
])

## パイプラインの確認
set_config(display='diagram')   
MMM_pipe

- **先程求めた最適なハイパーパラメータを設定**

In [ ]:
# パイプラインのハイパーパラメータの設定
best_params={
 'adstock__TVCM_pipe__carryover__func': 
    study.best_params.get('TVCM_Carryover_func'),
 'adstock__TVCM_pipe__carryover__length': 
    study.best_params.get('TVCM_length'),
 'adstock__TVCM_pipe__carryover__peak': 
    study.best_params.get('TVCM_peak'),
 'adstock__TVCM_pipe__carryover__rate': 
    study.best_params.get('TVCM_rate'),
 'adstock__TVCM_pipe__saturation__func': 
    study.best_params.get('TVCM_Saturation_func'),
 'adstock__TVCM_pipe__saturation__a': 
    study.best_params.get('TVCM_a'),
 'adstock__TVCM_pipe__saturation__K': 
    study.best_params.get('TVCM_K'),
 'adstock__TVCM_pipe__saturation__m': 
    study.best_params.get('TVCM_m'),
 'adstock__TVCM_pipe__saturation__b': 
    study.best_params.get('TVCM_b'),
 'adstock__TVCM_pipe__saturation__c': 
    study.best_params.get('TVCM_c'),
 'adstock__Newspaper_pipe__carryover__func': 
    study.best_params.get('Newspaper_Carryover_func'),
 'adstock__Newspaper_pipe__carryover__length': 
    study.best_params.get('Newspaper_length'),
 'adstock__Newspaper_pipe__carryover__peak': 
    study.best_params.get('Newspaper_peak'),
 'adstock__Newspaper_pipe__carryover__rate': 
    study.best_params.get('Newspaper_rate'),
 'adstock__Newspaper_pipe__saturation__func': 
    study.best_params.get('Newspaper_Saturation_func'),
 'adstock__Newspaper_pipe__saturation__a': 
    study.best_params.get('Newspaper_a'),
 'adstock__Newspaper_pipe__saturation__K': 
    study.best_params.get('Newspaper_K'),
 'adstock__Newspaper_pipe__saturation__m': 
    study.best_params.get('Newspaper_m'),
 'adstock__Newspaper_pipe__saturation__b': 
    study.best_params.get('Newspaper_b'),
 'adstock__Newspaper_pipe__saturation__c': 
    study.best_params.get('Newspaper_c'),
 'adstock__Web_pipe__carryover__func':
    study.best_params.get('Web_Carryover_func'),
 'adstock__Web_pipe__carryover__length': 
    study.best_params.get('Web_length'),
 'adstock__Web_pipe__carryover__peak': 
    study.best_params.get('Web_peak'),
 'adstock__Web_pipe__carryover__rate': 
    study.best_params.get('Web_rate'),
 'adstock__Web_pipe__saturation__func': 
    study.best_params.get('Web_Saturation_func'),
 'adstock__Web_pipe__saturation__a': 
    study.best_params.get('Web_a'),
 'adstock__Web_pipe__saturation__K': 
    study.best_params.get('Web_K'),
 'adstock__Web_pipe__saturation__m': 
    study.best_params.get('Web_m'),
 'adstock__Web_pipe__saturation__b': 
    study.best_params.get('Web_b'),
 'adstock__Web_pipe__saturation__c': 
    study.best_params.get('Web_c'),
}

### **精度検証**

In [ ]:
# パイプラインのインスタンス
MMM_pipe_best = MMM_pipe.set_params(**best_params)

In [ ]:
# クロスバリデーションで精度検証（R2）
np.mean(cross_val_score(MMM_pipe_best, 
                        X, y, 
                        cv=TimeSeriesSplit()
                       )
       )

In [ ]:
# 全データで精度検証（R2）
MMM_pipe_best.fit(X, y)
MMM_pipe_best.score(X, y)

- 線形回帰モデルなのか（切片と回帰係数）

In [ ]:
# 線形回帰モデルの切片と回帰係数
intercept = MMM_pipe_best.named_steps['regression'].intercept_ #切片
coef = MMM_pipe_best.named_steps['regression'].coef_ #回帰係数
# 回帰係数をデータフレーム化
weights = pd.Series(
    coef,
    index=X.columns
)
# 結果出力（切片と係数）
print('Intercept:\n', intercept, sep='')
print()
print('Coefficients:\n',weights, sep='')

### **売上貢献度の算出**
- **パイプラインの変換器（adstock）を抽出**

In [ ]:
# Piplineの変換器（adstock）を抽出
adstock = MMM_pipe_best.named_steps['adstock']

- **説明変数を変換し、学習データのインプットデータを作る**

In [ ]:
# 説明変数Xの変換
X_trans = pd.DataFrame(adstock.transform(X),
                       index=X.
                       index,columns=X.columns)

- **先程求めた線形回帰式を使い、売上貢献度（補正前）を計算**

In [ ]:
# 貢献度（補正前）
unadj_contribution = X_trans.mul(weights) #Xと係数を乗算
unadj_contribution = unadj_contribution.assign(Base=intercept) #切片の追加
unadj_contribution.head() #確認

- 週ごとに各媒体の売上貢献度を合計すると、**売上の予測値**

In [ ]:
# 貢献度の合計（yの予測値）
y_pred = unadj_contribution.sum(axis=1)
y_pred.head() #確認

In [ ]:
y.head() #確認

- **予測値と実測値が乖離**していることが分かります。この乖離をなくすために、**補正係数（correction factor）**を計算し、**売上貢献度を補正**

In [ ]:
# 補正係数
correction_factor = y.div(y_pred, axis=0)
correction_factor.head() #確認

- この**補正係数**を使い、**売上貢献度を補正**

In [ ]:
# 貢献度（補正後）
adj_contribution = (unadj_contribution
                    .mul(correction_factor, axis=0)
                   ) 
# 順番の変更
adj_contribution = adj_contribution[['Base', 'Web', 'Newspaper', 'TVCM']]
#確認
adj_contribution.head()

- **週×媒体別の売上貢献度**が求められたので、**積み上げグラフ**を作成

In [ ]:
# グラフ化
ax = (adj_contribution
      .plot.area(
          figsize=(16, 10),
          linewidth=0,
          ylabel='Sales',
          xlabel='Date')
     )
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[::-1], labels[::-1])

- **媒体別に全ての週の売上貢献度を合計**し、**媒体別の売上貢献度（円と構成比％）**と、そのグラフを作り、**何がどれほど売上に貢献したのか**

In [ ]:
# 媒体別の貢献度の合計
contribution_sum = adj_contribution.sum(axis=0)
#集計結果
print('売上貢献度（円）:\n', 
      contribution_sum, 
      sep=''
     )
print()
print('売上貢献度（構成比）:\n', 
      contribution_sum/contribution_sum.sum(), 
      sep=''
     )
#グラフ化
contribution_sum.plot.pie(fontsize=24)

- **費用対効果**を見てみます。今回は媒**体別にROIを計算**します。
- 先ず、**媒体別のコスト**を集計

In [ ]:
# 各媒体のコストの合計
cost_sum = X.sum(axis=0)
cost_sum #確認

- 先程求めた売上貢献度を使い、**媒体別のROIを計算しグラフ化**

In [ ]:
# 各媒体のROIの計算
ROI = (contribution_sum.drop('Base', axis=0) - cost_sum)/cost_sum
#確認
print('ROI:\n', ROI, sep='')
# グラフ化
ROI.plot.bar(fontsize=24)

- **ROI**は、値が大きいほど良く、少なくとも0以上である必要があります。
- TVCMは0以上なのはTVCMとWebです。Webが非常にいいということが分かります。一方、Newspaperは0未満なので、少なくとも止めたほうが良

***
<a id="RLE"></a>
# <p>**Ridge,Lasso,ElasticNet**</p>
- **広告・販促は似たような時期に集中し実施**されるので、変数間で相関が発生する。**この問題の解決として正則化項を持つ線形回帰を行う**
- モデルの説明力や予測精度は若干悪化するが、有効な手段
    - **線形回帰**
        - 通常の線形回帰モデルは、**最小二乗法**という方法で回帰式の**回帰係数を求める(定数含む)**
            - $y_{i}=\beta_{0}+/displaystyle\sum_{j=1}^{p}\beta_{j}x_{ij}$
            - yが目的変数で、xが説明変数。pが説明変数の数で、nがデータ数(データセットの行数)
                - 何かしら回帰係数(定数含む)を設定すると、何かしらの線形回帰式が出来上がる。何かしらの線形回帰式が出来上がれば、yの予測値を求めれる
                - **予測値と実測値の差を二乗した値を最小化**するような**回帰係数(定数含む)**を求める方法が、**最小二乗法**
                    - $\hat{\beta}=\underset{\beta}{\operatorname{argmin}}\lbrace \displaystyle\sum_{i=1}^{n}(\beta_{0}+\displaystyle\sum_{j=1}^{p}\beta_{j}x_{ij}-y_{j})^{2} \rbrace$
                - **最小二乗法にある制約(正則化項)**を付け加えた、**制約(正則化項)付き最小二乗法**という方法で回帰式の回帰係数(定数含む)を求めるのが、**正則化付き線形回帰モデル**
                    - 通常線形回帰モデルと比べ、求めた回帰係数(定数含む)が0の方向に縮小(絶対値が小さくなる)
                    - この**制約(正則化項)付き最小二乗法**という方法で回帰式の回帰係数(定数含む)を求めるのが、**正則化項付き線形回帰モデル**
                    - この**制約(正則化項)**により、説明変数同士の相関が高いために起こる不具合や過学習などの影響を緩和する。
                - [**Ridge回帰**](#ridge)
                    - **線形回帰モデルの回帰係数の二乗和を正則化項として加えた推定法**
                    - **ハイパーパラメータ**として、**正則化項の重み（正則化パラメータ）を設定**する必要
                        - $\hat{\beta}_{ridge}=\underset{\beta}{\operatorname{argmin}}\lbrace \displaystyle\sum_{i=1}^{n}(\beta_{0}+\displaystyle\sum_{j=1}^{p}\beta_{j}x_{ij}-y_{j})^{2}+\lambda \displaystyle\sum_{j=1}^{p}\beta_{j}^{2} \rbrace$
                - **Lasso回帰**
                    - **Ridge回帰と異なり**、**回帰係数を0にするため変数選択を自動で実施**
                    - **線形回帰モデルの回帰係数の絶対値の和を正則化項として加えた推定法**
                    - **ハイパーパラメータ**として、**正則化項の重み（正則化パラメータ）を設定**する必要
                        - $\hat{\beta}_{Lasso}=\underset{\beta}{\operatorname{argmin}}\lbrace \displaystyle\sum_{i=1}^{n}(\beta_{0}+\displaystyle\sum_{j=1}^{p}\beta_{j}x_{ij}-y_{j})^{2}+\lambda \displaystyle\sum_{j=1}^{p}|\beta_{j}| \rbrace$
                - **ElasticNet回帰**
                    - **ハイパーパラメータ**として、その**割合を設定**
                    - **ハイパーパラメータ**として、**正則化項の重み（正則化パラメータ）を設定**
                        - $\hat{\beta}_{elasticnet}=\underset{\beta}{\operatorname{argmin}}\lbrace \displaystyle\sum_{i=1}^{n}(\beta_{0}+\displaystyle\sum_{j=1}^{p}\beta_{j}x_{ij}-y_{j})^{2}+\lambda \displaystyle\sum_{j=1}^{p}(\alpha|\beta_{j}|+(1+\alpha)\beta_{j}^{2}) \rbrace$
                        
    - **方法**
        - **AdStockを考慮しないRidge回帰**
            - AdStockを考慮していないのでパイプラインは構築しない。
        - **シンプルなAdStock考慮**
        - **様々なAdStock考慮**
            - [**最適な組み合わせ**](#Add_all)
                - **飽和モデル** : **指数型飽和モデル・ロジスティック型飽和モデル・ゴンペルツ型飽和モデル**
                - **ラグモデル** : **定率減少型ラグ効果モデル・ピーク可変型ラグ効果モデル**

In [ ]:
from sklearn.linear_model import Ridge

In [ ]:
# Optunaハイパーパラメータ探索

# Ridge回帰のインスタンス生成
MMM=Ridge()

# 探索するハイパーパラメータ範囲の設定
params = {
 'alpha': UniformDistribution(0.01, 1e10),
}

# ハイパーパラメータ探索の設定
optuna_search = OptunaSearchCV(
    estimator=MMM,
    param_distributions=params,
    n_trials=1000,
    cv=TimeSeriesSplit(),
    random_state=123,
)

# 探索実施
optuna_search.fit(X, y)

# 探索結果
optuna_search.best_params_

- **最適ハイパーパラメータで学習**

In [ ]:
# 最適ハイパーパラメータで学習

# パイプラインのインスタンス
MMM_best = MMM.set_params(**optuna_search.best_params_)

# 全データで学習
MMM_best.fit(X, y)

# R2（決定係数）
MMM_best.score(X, y)

- **予測式の確認**

In [ ]:
# 予測式の確認

# 線形回帰モデルの切片と回帰係数
intercept = MMM_best.intercept_ #切片
coef = MMM_best.coef_           #回帰係数
alpha = MMM_best.alpha          #正則化パラメータ

# 回帰係数をSeries（シリーズ）化
weights = pd.Series(
    coef,
    index=X.columns
)

# 結果出力（切片と係数）
print('Intercept:\n', intercept, sep='')
print()
print('Coefficients:\n',weights, sep='')
print()
print('alpha:\n', alpha, sep='')

- **予測の実施**

In [ ]:
# 予測の実施

# 目的変数y（売上）の予測
pred = pd.DataFrame(MMM_best.predict(X),
                    index=X.index,
                    columns=['y'])
# 各媒体による売上の予測
## 値がすべて0の説明変数
X_ = X.copy()
X_.iloc[:,:]=0
## Base
pred['Base'] = MMM_best.predict(X_)
## 媒体
for i in range(len(X.columns)):
    X_.iloc[:,:]=0
    X_.iloc[:,i]=X.iloc[:,i]
    pred[X.columns[i]] = MMM_best.predict(X_) - pred['Base']
    
print(pred) #確認

- **貢献度の算定**

In [ ]:
# 予測値の補正
correction_factor = y.div(pred['y'], axis=0)   #補正係数
pred_adj = pred.mul(correction_factor, axis=0) #補正後の予測値
# 各媒体の貢献度だけ抽出
contribution = pred_adj[['Base', 'Web', 'Newspaper', 'TVCM']]
print(contribution.head()) #確認
# グラフ化
ax = (contribution
      .plot.area(
          ylabel='Sales',
          xlabel='Week')
     )
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[::-1], labels[::-1])

- **貢献度の構成比の算出**

In [ ]:
# 媒体別の貢献度の合計
contribution_sum = contribution.sum(axis=0)
# 集計結果の出力
print('売上貢献度（円）:\n', 
      contribution_sum, 
      sep=''
     )
print()
print('売上貢献度（構成比）:\n', 
      contribution_sum/contribution_sum.sum(), 
      sep=''
     )
# グラフ化
contribution_sum.plot.pie(fontsize=24)

- **マーケティングROIの算定**

In [ ]:
# マーケティングROIの算定

# 各媒体のコストの合計
cost_sum = X.sum(axis=0)

# 各媒体のROIの計算
ROI = (contribution_sum.drop('Base', axis=0) - cost_sum)/cost_sum
print('ROI:\n', ROI, sep='') #確認

# グラフ化
ROI.plot.bar(fontsize=24)

***
<a id="ridge"></a>
## <p>**Ridge + Optuna**</p>
- **正則化ハイパーパラメータを探査**

- **飽和モデル + 変換器**

In [ ]:
# 飽和モデル
def Saturation(X,a):
    return 1 - np.exp(-a*X)

## 飽和モデル：変換器
class ExponentialSaturation(BaseEstimator, TransformerMixin):
    
    # 初期化
    def __init__(self, a=1.0):
        self.a = a #ハイパーパラメータ
    
    # 学習
    def fit(self, X, y=None):
        return self
    
    # 処理（入力→出力）
    def transform(self, X):
        return Saturation(X,self.a)

- **ラグモデル + 変換器**

In [ ]:
# ラグ効果モデル
def Carryover(X: np.ndarray, rate, length):
    filter = (
        rate ** np.arange(length)
    ).reshape(-1, 1) 
    convolution = convolve2d(X, filter)
    if length > 1 : convolution = convolution[: -(length-1)]
    return convolution

## ラグモデル：変換器
class ExponentialCarryover(BaseEstimator, TransformerMixin):
    # 初期化
    def __init__(self, rate=0.5, length=1):
        self.rate = rate #ハイパーパラメータ
        self.length = length #ハイパーパラメータ
        
    # 学習        
    def fit(self, X, y=None):
        return self
    
    # 処理（入力→出力）
    def transform(self, X: np.ndarray):
        return Carryover(X, self.rate, self.length)

- **Pipeline構築**

In [ ]:
# Pipeline

# 変換器（adstock）
adstock = ColumnTransformer(
    [
     ('TVCM_pipe', Pipeline([
                           ('carryover', ExponentialCarryover()),
                           ('saturation', ExponentialSaturation())
     ]), ['TVCM']),
     ('Newspaper_pipe', Pipeline([
                           ('carryover', ExponentialCarryover()),
                           ('saturation', ExponentialSaturation())
     ]), ['Newspaper']),
     ('Web_pipe', Pipeline([
                           ('carryover', ExponentialCarryover()),
                           ('saturation', ExponentialSaturation())
     ]), ['Web']),
    ],
    remainder='passthrough'
)

## 変換器（adstock）→学習器（regression）
MMM_pipe = Pipeline([
    ('adstock', adstock),
    ('regression', Ridge())
])

## パイプラインの確認
set_config(display='diagram')   
MMM_pipe

- **Pipeline : ハイパーパラメータ探索**

In [ ]:
# ハイパーパラメータ探索

# 探索するハイパーパラメータ範囲の設定
params = {
    
 ## Ridge
 'regression__alpha':
    UniformDistribution(0.01, 10000),
    
 ## TVCM
 'adstock__TVCM_pipe__carryover__rate':
    UniformDistribution(0, 1),
 'adstock__TVCM_pipe__carryover__length':
    IntUniformDistribution(1, 6),
 'adstock__TVCM_pipe__saturation__a':
    UniformDistribution(0, 0.01),
    
 ## Newspaper
 'adstock__Newspaper_pipe__carryover__rate':
    UniformDistribution(0, 1),
 'adstock__Newspaper_pipe__carryover__length':
    IntUniformDistribution(1, 6),
 'adstock__Newspaper_pipe__saturation__a':
    UniformDistribution(0, 0.01),
    
 ## Web
 'adstock__Web_pipe__carryover__rate':
    UniformDistribution(0, 1),
 'adstock__Web_pipe__carryover__length':
    IntUniformDistribution(1, 6),
 'adstock__Web_pipe__saturation__a':
    UniformDistribution(0, 0.01),
}

# ハイパーパラメータ探索の設定
optuna_search = OptunaSearchCV(
    estimator=MMM_pipe,
    param_distributions=params,
    n_trials=2000,
    cv=TimeSeriesSplit(),
    random_state=0,
)

# 探索実施
optuna_search.fit(X, y)

# 探索結果
optuna_search.best_params_

- **モデルの構築と予測**
    - **最適ハイパーパラメータで学習**

In [ ]:
# 最適ハイパーパラメータで学習

# パイプラインのインスタンス
MMM_pipe_best = MMM_pipe.set_params(**optuna_search.best_params_)

# 全データで学習
MMM_pipe_best.fit(X, y)

# R2（決定係数）
MMM_pipe_best.score(X, y)

- **予測式の確認**

In [ ]:
# 予測式の確認

# 線形回帰モデルの切片と回帰係数
intercept = MMM_pipe_best.named_steps['regression'].intercept_ #切片
coef = MMM_pipe_best.named_steps['regression'].coef_ #回帰係数
alpha = MMM_pipe_best.named_steps['regression'].alpha #正則化パラメータ

# 回帰係数をSeries（シリーズ）化
weights = pd.Series(
    coef,
    index=X.columns
)

# 結果出力（切片と係数）
print('Intercept:\n', intercept, sep='')
print()
print('Coefficients:\n',weights, sep='')
print()
print('alpha:\n', alpha, sep='')

- **予測の実施**

In [ ]:
# 予測の実施

# 目的変数y（売上）の予測
pred = pd.DataFrame(MMM_pipe_best.predict(X),
                    index=X.index,
                    columns=['y'])

# 各媒体による売上の予測

## 値がすべて0の説明変数
X_ = X.copy()
X_.iloc[:,:]=0

## Base
pred['Base'] = MMM_pipe_best.predict(X_)

## 媒体
for i in range(len(X.columns)):
    X_.iloc[:,:]=0
    X_.iloc[:,i]=X.iloc[:,i]
    pred[X.columns[i]] = MMM_pipe_best.predict(X_) - pred['Base']
    
print(pred) #確認

- **貢献度とマーケティングROIの算定**
- **貢献度の算定**

In [ ]:
# 貢献度の算定

# 予測値の補正
correction_factor = y.div(pred['y'], axis=0)   #補正係数
pred_adj = pred.mul(correction_factor, axis=0) #補正後の予測値

# 各媒体の貢献度だけ抽出
contribution = pred_adj[['Base', 'Web', 'Newspaper', 'TVCM']]
print(contribution.head())

# グラフ化
ax = (contribution
      .plot.area(
          ylabel='Sales',
          xlabel='Week')
     )

handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[::-1], labels[::-1])

- **貢献度の構成比**

In [ ]:
# 貢献度の構成比

# 媒体別の貢献度の合計
contribution_sum = contribution.sum(axis=0)

# 集計結果の出力
print('売上貢献度（円）:\n', 
      contribution_sum, 
      sep=''
     )
print()
print('売上貢献度（構成比）:\n', 
      contribution_sum/contribution_sum.sum(), 
      sep=''
     )

# グラフ化
contribution_sum.plot.pie(fontsize=24)

- **マーケティングROI**

In [ ]:
# マーケティングROIの算定

# 各媒体のコストの合計
cost_sum = X.sum(axis=0)

# 各媒体のROIの計算
ROI = (contribution_sum.drop('Base', axis=0) - cost_sum)/cost_sum
print('ROI:\n', ROI, sep='') #確認

# グラフ化
ROI.plot.bar(fontsize=24)

***
<a id="Add_all"></a>
### <p>**最適な組み合わせ**</p>
- **飽和モデル** : **指数型飽和モデル・ロジスティック型飽和モデル・ゴンペルツ型飽和モデル**
- **ラグモデル** : **定率減少型ラグ効果モデル・ピーク可変型ラグ効果モデル**

- **飽和モデル**

In [ ]:
# 飽和モデル

# 指数型飽和モデル（exp_Saturation）
def exp_Saturation(X: np.ndarray,a):
    return 1 - np.exp(-a*X)

# ロジスティック型飽和モデル（logit_Saturation）
def logit_Saturation(X: np.ndarray,K,b,c,m):
    return K/(1+b*np.exp(-c*(X-m)))

# ゴンペルツ型飽和モデル（gom_Saturation）
def gom_Saturation(X: np.ndarray,K,b,c,m):
    return K*(b**np.exp(-c*(X-m)))

In [ ]:
# 変換器

# 指数型飽和モデル（exp_Saturation）
class pipe_exp_Saturation(BaseEstimator, TransformerMixin):
    
    # 初期化
    def __init__(self,a=1.0):
        self.a = a
    
    # 学習
    def fit(self, X, y=None):
        return self
    
    # 処理（入力→出力）
    def transform(self, X):
        X_ = exp_Saturation(X,
                            self.a
                           ).reshape(-1,1)
        return X_

# ロジスティック型飽和モデル（logit_Saturation）
class pipe_logit_Saturation(BaseEstimator, TransformerMixin):
    
    # 初期化
    def __init__(self,K=1.0,m=100.0,b=1.0, c=0.1):
        self.K = K
        self.m = m
        self.b = b
        self.c = c
    
    # 学習
    def fit(self, X, y=None):
        return self
    
    # 処理（入力→出力）
    def transform(self, X):
        X_ = logit_Saturation(X,
                              self.K, 
                              self.b, 
                              self.c, 
                              self.m
                             ).reshape(-1,1)
        return X_

# ゴンペルツ型飽和モデル（gom_Saturation）
class pipe_gom_Saturation(BaseEstimator, TransformerMixin):
    
    # 初期化
    def __init__(self,K=1.0,m=100.0,b=1.0,c=0.1):
        self.K = K
        self.m = m
        self.b = b
        self.c = c
    
    # 学習
    def fit(self, X, y=None):
        return self
    
    # 処理（入力→出力）
    def transform(self, X):
        X_ = gom_Saturation(X,
                            self.K, 
                            self.b, 
                            self.c, 
                            self.m
                           ).reshape(-1,1)
        return X_

- **ラグ効果モデル**

In [ ]:
# ラグ効果モデル

# 定率減少型ラグ効果モデル（simple_Carryover）
def simple_Carryover(X: np.ndarray, rate, length):
    
    filter = (
        rate ** np.arange(length)
    ).reshape(-1, 1) 
    
    convolution = convolve2d(X, filter)
    
    if length > 1 : convolution = convolution[: -(length-1)]
        
    return convolution

# ピーク可変型ラグ効果モデル（peak_Carryover）
def peak_Carryover(X: np.ndarray, length, peak, rate):
    X = np.append(np.zeros(length-1), X)
    
    Ws = np.zeros(length)
    
    for l in range(length):
        W = rate**((l-peak)**2)
        Ws[length-1-l] = W
    
    carryover_X = []
    
    for i in range(length-1, len(X)):
        X_array = X[i-length+1:i+1]
        Xi = sum(X_array * Ws)/sum(Ws)
        carryover_X.append(Xi)
        
    return np.array(carryover_X)

In [ ]:
# 変換器

# 定率減少型ラグ効果モデル　※ピークが広告など投入時（simple_Carryover）
class pipe_simple_Carryover(BaseEstimator, TransformerMixin):
    # 初期化
    def __init__(self,length=4,rate=0.5):
        self.length = length
        self.rate = rate
        
    # 学習        
    def fit(self, X, y=None):
        return self
    # 処理（入力→出力）
    def transform(self, X):
        X_ = simple_Carryover(X, 
                              self.rate, 
                              self.length
                             )
        return X_
    
# ピーク可変型ラグ効果モデル
class pipe_peak_Carryover(BaseEstimator, TransformerMixin):
    # 初期化
    def __init__(self,length=4,peak=1,rate=0.5):
        self.length = length
        self.peak = peak
        self.rate = rate
        
    # 学習        
    def fit(self, X, y=None):
        return self
    # 処理（入力→出力）
    def transform(self, X):
        X_ = peak_Carryover(X, 
                            self.length, 
                            self.peak, 
                            self.rate
                           ) 
        return X_

- **Optunaで利用する目的関数を定義**

In [ ]:
# 目的関数の設定
def objective(trial):
    
    #Ridge
    
    alpha = trial.suggest_float(
        "alpha",
        0.01, 10000
    )
    
    #TVCM
    ##TVCM Adstock func
    
    TVCM_Saturation_func = trial.suggest_categorical(
        "TVCM_Saturation_func",
        ["exp", "logit","gom"]
    )
    
    TVCM_Carryover_func = trial.suggest_categorical(
        "TVCM_Carryover_func",
        ["simple", "peak"]
    )
    
    ##TVCM_Saturation
    
    if TVCM_Saturation_func == 'exp':
        
        TVCM_a = trial.suggest_float(
            "TVCM_a", 
            0, 0.01
        )
        
        TVCM_Saturation = pipe_exp_Saturation(a=TVCM_a)
        
    elif TVCM_Saturation_func == 'logit':
        
        TVCM_K = trial.suggest_float(
            "TVCM_K", 
            np.min(X.TVCM), np.max(X.TVCM)*2
        )
        TVCM_m = trial.suggest_float(
            "TVCM_m",
            np.min(X.TVCM), np.max(X.TVCM)
        )
        TVCM_b = trial.suggest_float(
            "TVCM_b",
            0, 10
        )
        TVCM_c = trial.suggest_float(
            "TVCM_c",
            0, 1
        )
        
        TVCM_Saturation = pipe_logit_Saturation(
            K=TVCM_K,
            m=TVCM_m,
            b=TVCM_b,
            c=TVCM_c
        )
        
    else:
        
        TVCM_K = trial.suggest_float(
            "TVCM_K", 
            np.min(X.TVCM), 
            np.max(X.TVCM)*2
        )
        TVCM_m = trial.suggest_float(
            "TVCM_m", 
            np.min(X.TVCM), 
            np.max(X.TVCM)
        )
        TVCM_b = trial.suggest_float(
            "TVCM_b", 
            0, 1
        )
        TVCM_c = trial.suggest_float(
            "TVCM_c", 
            0, 1
        )     
        
        TVCM_Saturation = pipe_gom_Saturation(
            K=TVCM_K,
            m=TVCM_m,
            b=TVCM_b,
            c=TVCM_c
        )
        
    ##TVCM Carryover
    
    if TVCM_Carryover_func == 'simple':
        
        TVCM_rate = trial.suggest_float(
            "TVCM_rate", 
            0,1
        )
        TVCM_length = trial.suggest_int(
            "TVCM_length", 
            1,6
        )
        
        TVCM_Carryover = pipe_simple_Carryover(
            length=TVCM_length,
            rate=TVCM_rate
        )
        
    else:
        
        TVCM_rate = trial.suggest_float(
            "TVCM_rate", 
            0, 1
        )
        TVCM_length = trial.suggest_int(
            "TVCM_length",
            1,6
        )
        TVCM_peak = trial.suggest_int(
            "TVCM_peak",
            0,2
        )
        
        TVCM_Carryover = pipe_peak_Carryover(
            length=TVCM_length, 
            rate=TVCM_rate,
            peak=TVCM_peak
        )   
        
    #Newspaper
    ##Newspaper Adstock func
    
    Newspaper_Saturation_func = trial.suggest_categorical(
        "Newspaper_Saturation_func",
        ["exp", "logit","gom"]
    )
    
    Newspaper_Carryover_func = trial.suggest_categorical(
        "Newspaper_Carryover_func",
        ["simple", "peak"]
    )
    
    ##Newspaper_Saturation
    
    if Newspaper_Saturation_func == 'exp':
        
        Newspaper_a = trial.suggest_float(
            "Newspaper_a", 
            0, 0.01
        )
        
        Newspaper_Saturation = pipe_exp_Saturation(a=Newspaper_a)
        
    elif Newspaper_Saturation_func == 'logit':
        
        Newspaper_K = trial.suggest_float(
            "Newspaper_K",
            np.min(X.Newspaper), np.max(X.Newspaper)*2
        )
        Newspaper_m = trial.suggest_float(
            "Newspaper_m", 
            np.min(X.Newspaper), np.max(X.Newspaper)
        )
        Newspaper_b = trial.suggest_float(
            "Newspaper_b", 
            0, 10
        )
        Newspaper_c = trial.suggest_float(
            "Newspaper_c", 
            0, 1
        )
        
        Newspaper_Saturation = pipe_logit_Saturation(
            K=Newspaper_K,
            m=Newspaper_m,
            b=Newspaper_b,
            c=Newspaper_c
        )
        
    else:
        
        Newspaper_K = trial.suggest_float(
            "Newspaper_K", 
            np.min(X.Newspaper), np.max(X.Newspaper)*2
        )
        Newspaper_m = trial.suggest_float(
            "Newspaper_m", 
            np.min(X.Newspaper), np.max(X.Newspaper)
        )
        Newspaper_b = trial.suggest_float(
            "Newspaper_b", 
            0, 1
        )
        Newspaper_c = trial.suggest_float(
            "Newspaper_c", 
            0, 1
        )     
        
        Newspaper_Saturation = pipe_gom_Saturation(
            K=Newspaper_K,
            m=Newspaper_m,
            b=Newspaper_b,
            c=Newspaper_c
        )
        
    ##Newspaper Carryover
    
    if Newspaper_Carryover_func == 'simple':
        
        Newspaper_rate = trial.suggest_float(
            "Newspaper_rate", 
            0,1
        )
        Newspaper_length = trial.suggest_int(
            "Newspaper_length", 
            1,6
        )
        
        Newspaper_Carryover = pipe_simple_Carryover(
            length=Newspaper_length,
            rate=Newspaper_rate
        )
        
    else:
        
        Newspaper_rate = trial.suggest_float(
            "Newspaper_rate", 
            0, 1
        )
        Newspaper_length = trial.suggest_int(
            "Newspaper_length",
            1,6
        )
        Newspaper_peak = trial.suggest_int(
            "Newspaper_peak",
            0,2
        )
        
        Newspaper_Carryover = pipe_peak_Carryover(
            length=Newspaper_length,
            rate=Newspaper_rate,
            peak=Newspaper_peak
        )
            
    #Web
    ##Web Adstock func
    
    Web_Saturation_func = trial.suggest_categorical(
        "Web_Saturation_func",
        ["exp", "logit","gom"]
    )
    
    Web_Carryover_func = trial.suggest_categorical(
        "Web_Carryover_func",
        ["simple", "peak"]
    )
    
    ##Web_Saturation
    
    if Web_Saturation_func == 'exp':
        
        Web_a = trial.suggest_float(
            "Web_a", 
            0, 0.01
        )
        
        Web_Saturation = pipe_exp_Saturation(a=Web_a)
        
    elif Web_Saturation_func == 'logit':
        
        Web_K = trial.suggest_float(
            "Web_K", 
            np.min(X.Web), np.max(X.Web)*2
        )
        Web_m = trial.suggest_float(
            "Web_m", 
            np.min(X.Web), np.max(X.Web)
        )
        Web_b = trial.suggest_float(
            "Web_b", 
            0, 10
        )
        Web_c = trial.suggest_float(
            "Web_c", 
            0, 1
        )
        
        Web_Saturation = pipe_logit_Saturation(
            K=Web_K,
            m=Web_m,
            b=Web_b,
            c=Web_c
        )
        
    else:
        
        Web_K = trial.suggest_float(
            "Web_K", 
            np.min(X.Web), np.max(X.Web)*2
        )
        Web_m = trial.suggest_float(
            "Web_m", 
            np.min(X.Web), np.max(X.Web)
        )
        Web_b = trial.suggest_float(
            "Web_b", 
            0, 1
        )
        Web_c = trial.suggest_float(
            "Web_c", 
            0, 1
        )     
        
        Web_Saturation = pipe_gom_Saturation(
            K=Web_K,
            m=Web_m,
            b=Web_b,
            c=Web_c
        )
        
    ##Web Carryover
    
    if Web_Carryover_func == 'simple':
        
        Web_rate = trial.suggest_float(
            "Web_rate", 
            0,1
        )
        Web_length = trial.suggest_int(
            "Web_length", 
            1,6
        )
        
        Web_Carryover = pipe_simple_Carryover(
            length=Web_length,
            rate=Web_rate
        )
        
    else:
        
        Web_rate = trial.suggest_float(
            "Web_rate", 
            0, 1
        )
        Web_length = trial.suggest_int(
            "Web_length",
            1,6
        )
        Web_peak = trial.suggest_int(
            "Web_peak",
            0,2
        )
        
        Web_Carryover = pipe_peak_Carryover(
            length=Web_length,
            rate=Web_rate,
            peak=Web_peak
        )   
    
    # パイプライン化
    ## 変換器（Adstock）
    adstock = ColumnTransformer(
        [
         ('TVCM_pipe', Pipeline([
             ('TVCM_carryover', TVCM_Carryover),
             ('TVCM_saturation', TVCM_Saturation)
         ]), ['TVCM']),
         ('Newspaper_pipe', Pipeline([
             ('Newspaper_carryover', Newspaper_Carryover),
             ('Newspaper_saturation', Newspaper_Saturation)
         ]), ['Newspaper']),
         ('Web_pipe', Pipeline([
             ('Web_carryover', Web_Carryover),
             ('Web_saturation', Web_Saturation)
         ]), ['Web']),
        ],
        remainder='passthrough'
    )
    
    ## 説明変数の変換（adstock）→線形回帰モデル（regression）
    MMM_pipe = Pipeline([
        ('adstock', adstock),
        ('regression', Ridge(alpha=alpha))
    ])
        
    #CVによる評価
    score = cross_val_score(
        MMM_pipe,
        X,
        y,
        cv=TimeSeriesSplit()
    )
    accuracy = score.mean()
    return accuracy

In [ ]:
# 目的関数の最適化を実行する
study = optuna.create_study(direction="maximize")
study.optimize(objective,
               n_trials=10000,
               show_progress_bar=True)

In [ ]:
# 探索結果
study.best_params

- **モデルの構築と予測**
- **パイプライン構築**
    - パイプライン用の、飽和モデルとキャリーオーバー効果の2つの変換器クラスを定義

In [ ]:
# Pipeline用変換器

## 飽和モデル
class Saturation(BaseEstimator, TransformerMixin):
    
    # 初期化
    def __init__(self,func='exp',a=1.0,K=1.0,m=100.0,b=1.0, c=0.1):
        self.func = func
        self.a = a
        self.K = K
        self.m = m
        self.b = b
        self.c = c
    
    # 学習
    def fit(self, X, y=None):
        return self
    
    # 処理（入力→出力）
    def transform(self, X):
        if self.func == 'exp':
            X_ = exp_Saturation(X,
                                self.a
                               ).reshape(-1,1)
        elif self.func == "logit":
            X_ = logit_Saturation(X,
                                  self.K, 
                                  self.b,
                                  self.c, 
                                  self.m
                                 ).reshape(-1,1)
        else:
            X_ = gom_Saturation(X,
                                self.K, 
                                self.b, 
                                self.c, 
                                self.m
                               ).reshape(-1,1)
        return X_
    
## ラグ効果モデル
class Carryover(BaseEstimator, TransformerMixin):
    # 初期化
    def __init__(self,func='simple',length=4, peak=1, rate=0.5):
        self.func = func
        self.length = length
        self.peak = peak
        self.rate = rate
        
    # 学習        
    def fit(self, X, y=None):
        return self
    # 処理（入力→出力）
    def transform(self, X):
        if self.func == 'peak':
            X_ = peak_Carryover(X, 
                                self.length, 
                                self.peak, 
                                self.rate)  
        else:
            X_ = simple_Carryover(X, 
                                  self.rate, 
                                  self.length)
            
        return X_

- **2つの変換器でアドストック（Ad Stock）**を表現します。これらの**変換器と学習器（Ridge回帰モデル）をつなげたパイプラインを構築**

In [ ]:
# Pipelineの設定

## 説明変数の変換部分（adstock）の定義
adstock = ColumnTransformer(
    [
     ('TVCM_pipe', Pipeline([
                           ('carryover', Carryover()),
                           ('saturation', Saturation())
     ]), ['TVCM']),
     ('Newspaper_pipe', Pipeline([
                           ('carryover', Carryover()),
                           ('saturation', Saturation())
     ]), ['Newspaper']),
     ('Web_pipe', Pipeline([
                           ('carryover', Carryover()),
                           ('saturation', Saturation())
     ]), ['Web']),
    ],
    remainder='passthrough'
)

## 説明変数の変換（adstock）→線形回帰モデル（regression）(Ridge使用)
MMM_pipe = Pipeline([
    ('adstock', adstock),
    ('regression', Ridge())
])
## パイプラインの確認
set_config(display='diagram')   
MMM_pipe

- **最適ハイパーパラメータで学習**

In [ ]:
# 最適ハイパーパラメータ
best_params={
 'regression__alpha': 
    study.best_params.get('alpha'),
 'adstock__TVCM_pipe__carryover__func': 
    study.best_params.get('TVCM_Carryover_func'),
 'adstock__TVCM_pipe__carryover__length': 
    study.best_params.get('TVCM_length'),
 'adstock__TVCM_pipe__carryover__peak': 
    study.best_params.get('TVCM_peak'),
 'adstock__TVCM_pipe__carryover__rate': 
    study.best_params.get('TVCM_rate'),
 'adstock__TVCM_pipe__saturation__func': 
    study.best_params.get('TVCM_Saturation_func'),
 'adstock__TVCM_pipe__saturation__a': 
    study.best_params.get('TVCM_a'),
 'adstock__TVCM_pipe__saturation__K': 
    study.best_params.get('TVCM_K'),
 'adstock__TVCM_pipe__saturation__m': 
    study.best_params.get('TVCM_m'),
 'adstock__TVCM_pipe__saturation__b': 
    study.best_params.get('TVCM_b'),
 'adstock__TVCM_pipe__saturation__c': 
    study.best_params.get('TVCM_c'),
 'adstock__Newspaper_pipe__carryover__func': 
    study.best_params.get('Newspaper_Carryover_func'),
 'adstock__Newspaper_pipe__carryover__length': 
    study.best_params.get('Newspaper_length'),
 'adstock__Newspaper_pipe__carryover__peak': 
    study.best_params.get('Newspaper_peak'),
 'adstock__Newspaper_pipe__carryover__rate': 
    study.best_params.get('Newspaper_rate'),
 'adstock__Newspaper_pipe__saturation__func': 
    study.best_params.get('Newspaper_Saturation_func'),
 'adstock__Newspaper_pipe__saturation__a': 
    study.best_params.get('Newspaper_a'),
 'adstock__Newspaper_pipe__saturation__K': 
    study.best_params.get('Newspaper_K'),
 'adstock__Newspaper_pipe__saturation__m': 
    study.best_params.get('Newspaper_m'),
 'adstock__Newspaper_pipe__saturation__b': 
    study.best_params.get('Newspaper_b'),
 'adstock__Newspaper_pipe__saturation__c': 
    study.best_params.get('Newspaper_c'),
 'adstock__Web_pipe__carryover__func':
    study.best_params.get('Web_Carryover_func'),
 'adstock__Web_pipe__carryover__length': 
    study.best_params.get('Web_length'),
 'adstock__Web_pipe__carryover__peak': 
    study.best_params.get('Web_peak'),
 'adstock__Web_pipe__carryover__rate': 
    study.best_params.get('Web_rate'),
 'adstock__Web_pipe__saturation__func': 
    study.best_params.get('Web_Saturation_func'),
 'adstock__Web_pipe__saturation__a': 
    study.best_params.get('Web_a'),
 'adstock__Web_pipe__saturation__K': 
    study.best_params.get('Web_K'),
 'adstock__Web_pipe__saturation__m': 
    study.best_params.get('Web_m'),
 'adstock__Web_pipe__saturation__b': 
    study.best_params.get('Web_b'),
 'adstock__Web_pipe__saturation__c': 
    study.best_params.get('Web_c'),
}

# パイプラインのインスタンス
MMM_pipe_best = MMM_pipe.set_params(**best_params)

# 全データで学習
MMM_pipe_best.fit(X, y)

# R2（決定係数）
MMM_pipe_best.score(X, y)

- **予測の実施**

In [ ]:
# 予測の実施

# 目的変数y（売上）の予測
pred = pd.DataFrame(MMM_pipe_best.predict(X),
                    index=X.index,
                    columns=['y'])

# 各媒体による売上の予測
## 値がすべて0の説明変数
X_ = X.copy()
X_.iloc[:,:]=0

## Base
pred['Base'] = MMM_pipe_best.predict(X_)

## 媒体
for i in range(len(X.columns)):
    X_.iloc[:,:]=0
    X_.iloc[:,i]=X.iloc[:,i]
    pred[X.columns[i]] = MMM_pipe_best.predict(X_) - pred['Base']
    
print(pred) #確認

- **貢献度とマーケティングROIの算定**

In [ ]:
# 貢献度の算定

# 予測値の補正
correction_factor = y.div(pred['y'], axis=0)   #補正係数
pred_adj = pred.mul(correction_factor, axis=0) #補正後の予測値

# 各媒体の貢献度だけ抽出
contribution = pred_adj[['Base', 'Web', 'Newspaper', 'TVCM']]
print(contribution.head())

# グラフ化
ax = (contribution
      .plot.area(
          ylabel='Sales',
          xlabel='Week')
     )
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[::-1], labels[::-1])

- **貢献度の構成比の算出**

In [ ]:
# 貢献度の構成比の算出

# 媒体別の貢献度の合計
contribution_sum = contribution.sum(axis=0)

# 集計結果の出力
print('売上貢献度（円）:\n', 
      contribution_sum, 
      sep=''
     )
print()
print('売上貢献度（構成比）:\n', 
      contribution_sum/contribution_sum.sum(), 
      sep=''
     )

# グラフ化
contribution_sum.plot.pie(fontsize=24)

- **マーケティングROIの算定**

In [ ]:
# 各媒体のコストの合計
cost_sum = X.sum(axis=0)

# 各媒体のROIの計算
ROI = (contribution_sum.drop('Base', axis=0) - cost_sum)/cost_sum
print('ROI:\n', ROI, sep='')

# グラフ化
ROI.plot.bar(fontsize=24)

***
<a id="PCR"></a>
## <p>**主成分分析回帰(PCR)モデル**</p>
- 多くの広告・販促は一定の期間に集中し実施する事が多いので、相関が発生し問題が発生する。この対策の一つとして正則化付き線形回帰と**主成分分析回帰(PCR)**を使用する
    - **主成分分析回帰(PCR)**、**元の説明変数X**に対し**主成分分析**を実施し、**主成分得点(変換後のデータ)**を求め、その**主成分得点**を使い、**線形回帰モデル**を構築する手法。
    - 要は、**主成分分析×線形回帰**という**２つの数理モデルを組み合わせたもの** が、主成分回帰(PCR)
- **主成分分析(PCR)について**
    - PCAは訓練セットの分散を最大限に維持する軸を見つけ出す。
        - 訓練セットの主成分を見つける為には、訓練セット行列$XをU W V^{T}$の3行列のドット積に分解できる
        - **特異値分解(SVD：singular value decomposition)**という**標準的な行列分解(matrix factorization)テクニック**ここで、Vにはすべての主成分を定義する単位ベクトルを含む
        - **PCAは、データセットが原点を中心としてセンタリングされていることを前提としている。**
            - **scikit-learnのPCAは自動でデータのセンタリングをしてくれる**。他のライブラリを使用する場合には、データのセンタリングが自動かどうかに注意する。
        - 全ての主成分がみつかったら、最初のd個の成分が定義する超平面に射影すれば、データセットをd次元に**次元削減**できる。
            - これで超平面を選択すれば、射影しても分散が最大限に維持されることが保証される。
    - **因子寄与率**
        - **個々の主成分に沿ったデータセットの分散の分散全体に対する割合**
        - $from sklearn.decomposition import PCA$では、$explained_variance_ratio_ $属性から、個々の主成分の**因子寄与率(explained variance ratio)**が得られる。
            - $PCA(n_components=2)$の場合、2次元への射影になるので、因子寄与率は2であり、各値はデータセットの分散を示す。
    - **適切な次数の選択**
        - **次数をいくつまで削減するか**を無作為に選択するよりも、各次元に沿った因子寄与率の合計が十分な割合(例：95%)になるように次数を選択する。
            - 可視化の為の次数の選択であれば2か3次元でよい。(スイスロール)
            - 分散を維持する際に、オリジナルのデータセットのサイズが減る点に注意する。これは**圧縮**
                - **圧縮(率)**によって分類アルゴリズム(SVMなど)が大幅にスピードアップされる
            - **95%の分散を維持する**
                - $cumsum = np.cumsum(pca.explained_variance_ratio_)$
                - $d = np.argmax(cumsum >= 0.95) + 1$
            - **因子寄与率のプロット**
                - 上記の$cumsum$をプロットする
                    - 因子寄与率の伸びが鈍化する屈曲点があるので、これがそのデータセットの本来の次数だと考えられる。
            - **PCA射影の逆変換**
                - 逆変換を行う事によって、次元削減されたデータセットを元の次元サイズに再構築することも可能。
                    - 95%は5%のデータを削除しているので、元のデータが戻るわけではない。この５％は消失している。**元のデータに近いものが得られる**
                    - オリジナルデータと再構築されたデータの平均二乗距離を**再構築誤差(reconstruction error)**
                    - $pca = PCA(n_components=150)$
                    - $X_reduced = pca.fit_transform(X_train)$
                    - $X_recovered = pca.inverse_transform(X_reduced)$ ← **再構築誤差**実行
- **主成分分析(PCR)**を挟む理由
    - **データセットの次元を削減する**
    - **主成分同士の相関が小さい**
        - **主成分分析**を実施することで、**元の説明変数Xを主成分に変換**する。
            - **主成分分析**によって生み出された**主成分**は、**お互いの相関は非常に小さいもの**となる
        - **説明変数Xの変数の数**より少なくて済む。これが**次元削減**
- **主成分分析(PCR)の種類**
    - **ランダム化PCA**
        - **scikit-learnのsvd_solver="randomized"**
        - 最初のd個の主成分の概数を高速に見つけ出してくる**ランダム化PCA**確立的アルゴリズムを使用。
        - $O(m×n^{2})+O(n^{3})だが、ランダム化PCAはO(m×d^{2})+O(d^{3})$である為、dがnよりもかなり小さいときには大幅に高速になる
            - $rand_pca = PCA(n_components=150, svd_solver="randomized")$
            - デフォルトは$svd_solver="auto"$になっている。ｍかｎが500よりも大きく、ｄがｍかｎの80%よりも小さければ、自動的にランダム化PCAを使用する
            - **特異値分解**を強制する場合には$svd_solver="full"$っを指定する
    - **逐次学習型PCA(IPCA)**
        - 通常のPCAの実装には、**訓練セット全体がメモリに収まっていなければアルゴリズムを実行不可能**という問題がある。
        - **訓練セットをミニバッチに分解し、1度に１つずつミニバッチを渡す**
            - **大規模な訓練セットを使用する場合**や**PCAをオンライン実行(新インスタンス時の実行)**したい場合に有効
            - $IncrementalPCA$
            - Numpyのmemmapを使用。
                - ディスク上のバイナリファイルに格納された大規模な配列がまるでメモリ内にあるかのように操作する。
                - データが必要になった時に必要なだけのデータをメモリにロードする
                - $memm = np.memmap(filename, dtype="float32", mode="readonly", shape=(m, n))$
                - $batch_size = m // n_batches$
                - $inc_pca = IncrementalPCA(n_components=150, batch_size=batch_size)$
                - $inc_pca.fit(memm)$
    - **カーネルPCA**
        - **インスタンスを特徴量空間**にマッピング。**カーネルトリック**を使用。
        - カーネルトリックを使用すれば、PCAが次元削減のために**複雑な非線形射影**を実行できる
        - kPCAは、射影後にもインスタンスのクラスタをうまく保存でき、曲がりくねった多様体に近接するデータセットの展開にも使える
            - **ガウスRBFカーネル**
                - $from sklearn.decomposition import KernelPCA$
                - $rbf_pca = KernelPCA(n_components=2, kernel="rbf", gamma=0.04)$
                - $X_reduced = rbf_pca.fit_transform(X)$
            - **カーネルの選択とハイパーパラメータ調整**
                - **Optuna**を使用するのが早い
                - Otptunaを使用できない環境であれば**GridSearchやRand**を使用する。
                - **kernel**
                    - $"rbf", "kernel"$
            - **再構築プレイメージ**
                - 再構築実行
                    - $rbf_pca = KernelPCA(n_components=2, kernel="rbf", gamma=0.04, fit_inverse_transform=True)$
                    - $rbf_pca.fit_transform(X)$
                    - $rbf_pca.inverse_transform(X_reduced)$
    - **LLE(局所線形埋め込み法)**
        - 強力な**非線形次元削減(NLDR：nonlinear dimentionality reducation)**
        - 他のアルゴリズムとは異なり、**射影に依存しない多様体学習テクニック**
            - 個々の訓練インスタンスが最近傍インスタンスと線形にどのような関係になるかを測定し、局所的な関係がもっとも保存される訓練セットの低次元表現を探す。
            - **特にノイズがあまり多くない時には、曲がりくねった多様体の展開で力を発揮する**
                - $LocallyLinearEmbedding(n_components=2, n_neighbors=10)$
                - LLEの最初のステップは、**制約付き最適化問題**
            - **LLEの計算量**
                - k個の最近傍インスタンスを見つける為に、$O(m \log(m)n \log(k))$
                - 重みの最適化のために$O(mnk^{3})$
                - 低次元表現の構築のため$O(dm^{2})$、$m^{2}$で大規模なデータセットへのスケーラビリティは低い
    - **ランダム射影**
        - **ランダム線形射影**
        - 次元削減の品質はインスタンス数とターゲット次元によって左右されるが、初期次元の影響を受けない
    - **多次元尺度法(MDS)**
        - インスタンス間の距離を維持しようとしながら次元削減を行う
    - **Isomap**
        - 個々のインスタンスと複数の最近棒インスタンスを結んでグラフを作り、インスタンス間の**測地距離**を可能な限り維持しながら次元を削減する
    - **t-SNE**
        - t分布確率的近傍埋め込み法は、類似するインスタンスを近くに保ち、似ていないインスタンスを遠くに遠ざけようとしながら次元を削減する。
        - **可視化、特に高次空間のインスタンスのクラスタを視覚化するために使われることが多い。(MNISTの2次元可視化のような)**
    - **LDA**
        - これは**本来分類アルゴリズム**だが、訓練中にはクラスをもっとも特徴的に分ける軸を学習し、それらの軸を使ってデータを射影する超空間を定義する。
        - このため、**LDAはSVM分類器などその他の分類アルゴリズムを実行する前の次元削減テクニックとして優れている**
- **主成分回帰(PCR)のハイパーパラメータ**
    - **主成分数**
        - 主成分数を決める方法は様々ある。
- **主成分分析(PCR)モデルのデメリット**
    - モデルの説明力や予測精度は悪化する
- [**主成分分析を使用した、AdStockでの最適な組み合わせ**](#rag_sat)
    - **ラグモデル**
        - 定率減少型ラグ効果モデル（simple_Carryover）
        - ピーク可変型ラグ効果モデル（peak_Carryover）
    - **飽和モデル**
        - 指数型飽和モデル（exp_Saturation）
        - ロジスティック型飽和モデル（logit_Saturation）
        - ゴンペルツ型飽和モデル（gom_Saturation）

In [ ]:
!pip install optuna

In [ ]:
import optuna
import numpy as np
import pandas as pd

from scipy.signal import convolve2d

# Base
from sklearn.base import BaseEstimator, TransformerMixin

# Scaling
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer

# Model
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression

# SKl
from sklearn.pipeline import Pipeline
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
from sklearn import set_config

# Optuna
from optuna.integration import OptunaSearchCV
from optuna.distributions import IntUniformDistribution, UniformDistribution, IntDistribution, FloatDistribution

# Plot
import matplotlib.pyplot as plt
plt.style.use('ggplot') #グラフスタイル
plt.rcParams['figure.figsize'] = [12, 9] # グラフサイズ

In [ ]:
# DataLoad
url = 'https://www.salesanalytics.co.jp/4zdt'
df = pd.read_csv(url,
                 parse_dates=['Week'],
                 index_col='Week'
                )

# DataInfo
print(df.info()) #変数の情報
print(df.head()) #データの一部

# 説明変数Xと目的変数yに分解
X = df.drop(columns=['Sales'])
y = df['Sales']

***
### **基礎 : PCR**
- **パイプライン化**
- **主成分回帰（PCR）**は、
    - 元の説明変数Xを標準化し
    - 標準化した値を利用し主成分分析し
        - 標準化とは、元のデータを平均で引き標準偏差で割ったもの
        - sklearn : StandardScaler
    - 主成分得点と目的変数yで線形回帰モデルを構築
- **パイプライン構築**
    - **標準化(StandardScaler)**
    - **主成分分析(PCA)**
    - **Model：線形回帰(LinearRegression)**

In [ ]:
# Pipeline構築

# 標準化→主成分分析→線形回帰
MMM = Pipeline([
    ('SS', StandardScaler()), # < PCAが既にセンタリングするのにいるのか？
    ('PCA', PCA()),
    ('regression', LinearRegression())
])

## パイプラインの確認
set_config(display='diagram')   
MMM

- **ハイパーパラメータ探索**
- **主成分回帰（PCR）**の最適な**主成分の数**の値を探索；

In [ ]:
# ハイパーパラメータ探索
#
# 探索するハイパーパラメータ範囲の設定
params = {
 'PCA__n_components':IntUniformDistribution(1, 3), # IntDistribution()
}
# ハイパーパラメータ探索の設定
optuna_search = OptunaSearchCV(
    estimator=MMM,
    param_distributions=params,
    n_trials=1000,
    cv=TimeSeriesSplit(),
    random_state=123,
)
# 探索実施
optuna_search.fit(X, y)
# 探索結果
optuna_search.best_params_

- 最適な**主成分の数**
- **主成分の数**に設定し、MMMを全データで構築
- **最適ハイパーパラメータで学習**

In [ ]:
# 最適ハイパーパラメータで学習

# パイプラインのインスタンス
MMM_best = MMM.set_params(**optuna_search.best_params_)
# 全データで学習
MMM_best.fit(X, y)
# R2（決定係数）
MMM_best.score(X, y)

- **予測の実施**

In [ ]:
# 予測の実施

# 目的変数y（売上）の予測
pred = pd.DataFrame(MMM_best.predict(X),
                    index=X.index,
                    columns=['y'])

# 各媒体による売上の予測
## 値がすべて0の説明変数
X_ = X.copy()
X_.iloc[:,:]=0

## Base
pred['Base'] = MMM_best.predict(X_)

## 媒体
for i in range(len(X.columns)):
    X_.iloc[:,:]=0
    X_.iloc[:,i]=X.iloc[:,i]
    pred[X.columns[i]] = MMM_best.predict(X_) - pred['Base']
    
print(pred) #確認

- 貢献度とマーケティングROIの算定
- 貢献度の算定

In [ ]:
# 貢献度の算定

# 予測値の補正
correction_factor = y.div(pred['y'], axis=0)   #補正係数
pred_adj = pred.mul(correction_factor, axis=0) #補正後の予測値

# 各媒体の貢献度だけ抽出
contribution = pred_adj[['Base', 'Web', 'Newspaper', 'TVCM']]
print(contribution.head()) #確認

# グラフ化
ax = (contribution
      .plot.area(
          ylabel='Sales',
          xlabel='Week')
     )

handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[::-1], labels[::-1])

- **貢献度の構成比の算出**

In [ ]:
# 貢献度の構成比

# 媒体別の貢献度の合計
contribution_sum = contribution.sum(axis=0)

# 集計結果の出力
print('売上貢献度（円）:\n', 
      contribution_sum, 
      sep=''
     )
print()
print('売上貢献度（構成比）:\n', 
      contribution_sum/contribution_sum.sum(), 
      sep=''
     )

# グラフ化
contribution_sum.plot.pie(fontsize=24)

- **マーケティングROIの算定**

In [ ]:
# マーケティングROIの算定
#
# 各媒体のコストの合計
cost_sum = X.sum(axis=0)
# 各媒体のROIの計算
ROI = (contribution_sum.drop('Base', axis=0) - cost_sum)/cost_sum
print('ROI:\n', ROI, sep='') #確認
# グラフ化
ROI.plot.bar(fontsize=24)

***
<a id="rag_sat"></a>
## <p>**最適な組み合わせを使用したAdStock**</p>
- **ハイパーパラメータ探索**
- **主成分回帰モデル（PCR）**の最適な**主成分の数**と、**ラグ効果**や**飽和効果**の**ハイパーパラメータ**を探索

- **飽和モデル + 変換器**

In [ ]:
# 飽和モデル

### 指数型飽和モデル（exp_Saturation）
def exp_Saturation(X: np.ndarray,a):
    return 1 - np.exp(-a*X)

### ロジスティック型飽和モデル（logit_Saturation）
def logit_Saturation(X: np.ndarray,K,b,c,m):
    return K/(1+b*np.exp(-c*(X-m)))

### ゴンペルツ型飽和モデル（gom_Saturation）
def gom_Saturation(X: np.ndarray,K,b,c,m):
    return K*(b**np.exp(-c*(X-m)))

In [ ]:
## 変換器


### 指数型飽和モデル（exp_Saturation）
class pipe_exp_Saturation(BaseEstimator, TransformerMixin):
    
    # 初期化
    def __init__(self,a=1.0):
        self.a = a
    
    # 学習
    def fit(self, X, y=None):
        return self
    
    # 処理（入力→出力）
    def transform(self, X):
        X_ = exp_Saturation(X,
                            self.a
                           ).reshape(-1,1)
        return X_


### ロジスティック型飽和モデル（logit_Saturation）
class pipe_logit_Saturation(BaseEstimator, TransformerMixin):
    
    # 初期化
    def __init__(self,K=1.0,m=100.0,b=1.0, c=0.1):
        self.K = K
        self.m = m
        self.b = b
        self.c = c
    
    # 学習
    def fit(self, X, y=None):
        return self
    
    # 処理（入力→出力）
    def transform(self, X):
        X_ = logit_Saturation(X,
                              self.K, 
                              self.b, 
                              self.c, 
                              self.m
                             ).reshape(-1,1)
        return X_


### ゴンペルツ型飽和モデル（gom_Saturation）
class pipe_gom_Saturation(BaseEstimator, TransformerMixin):
    
    # 初期化
    def __init__(self,K=1.0,m=100.0,b=1.0,c=0.1):
        self.K = K
        self.m = m
        self.b = b
        self.c = c
    
    # 学習
    def fit(self, X, y=None):
        return self
    
    # 処理（入力→出力）
    def transform(self, X):
        X_ = gom_Saturation(X,
                            self.K, 
                            self.b, 
                            self.c, 
                            self.m
                           ).reshape(-1,1)
        return X_

- **ラグモデル+変換器**

In [ ]:
# ラグ効果モデル


### 定率減少型ラグ効果モデル（simple_Carryover）
def simple_Carryover(X: np.ndarray, rate, length):
    
    filter = (
        rate ** np.arange(length)
    ).reshape(-1, 1) 
    
    convolution = convolve2d(X, filter)
    
    if length > 1 : convolution = convolution[: -(length-1)]
        
    return convolution


### ピーク可変型ラグ効果モデル（peak_Carryover）
def peak_Carryover(X: np.ndarray, length, peak, rate):
    X = np.append(np.zeros(length-1), X)
    
    Ws = np.zeros(length)
    
    for l in range(length):
        W = rate**((l-peak)**2)
        Ws[length-1-l] = W
    
    carryover_X = []
    
    for i in range(length-1, len(X)):
        X_array = X[i-length+1:i+1]
        Xi = sum(X_array * Ws)/sum(Ws)
        carryover_X.append(Xi)
        
    return np.array(carryover_X)

In [ ]:
## 変換器の定義


### 定率減少型ラグ効果モデル　※ピークが広告など投入時（simple_Carryover）
class pipe_simple_Carryover(BaseEstimator, TransformerMixin):
    # 初期化
    def __init__(self,length=4,rate=0.5):
        self.length = length
        self.rate = rate
        
    # 学習        
    def fit(self, X, y=None):
        return self
    # 処理（入力→出力）
    def transform(self, X):
        X_ = simple_Carryover(X, 
                              self.rate, 
                              self.length
                             )
        return X_


### ピーク可変型ラグ効果モデル（peak_Carryover）
class pipe_peak_Carryover(BaseEstimator, TransformerMixin):
    # 初期化
    def __init__(self,length=4,peak=1,rate=0.5):
        self.length = length
        self.peak = peak
        self.rate = rate
        
    # 学習        
    def fit(self, X, y=None):
        return self
    # 処理（入力→出力）
    def transform(self, X):
        X_ = peak_Carryover(X, 
                            self.length, 
                            self.peak, 
                            self.rate
                           ) 
        return X_

- **Optuna**で利用する**目的関数**を定義する

In [ ]:
# 目的関数の設定
def objective(trial):
    
    #PCA
    n_components = trial.suggest_int(
        "n_components",
        1, 3
    )
    
    #TVCM
    
    ##TVCM Adstock func
    
    TVCM_Saturation_func = trial.suggest_categorical(
        "TVCM_Saturation_func",
        ["exp", "logit","gom"]
    )
    
    TVCM_Carryover_func = trial.suggest_categorical(
        "TVCM_Carryover_func",
        ["simple", "peak"]
    )
    
    ##TVCM_Saturation
    
    if TVCM_Saturation_func == 'exp':
        
        TVCM_a = trial.suggest_float(
            "TVCM_a", 
            0, 0.01
        )
        
        TVCM_Saturation = pipe_exp_Saturation(a=TVCM_a)
        
    elif TVCM_Saturation_func == 'logit':
        
        TVCM_K = trial.suggest_float(
            "TVCM_K", 
            np.min(X.TVCM), np.max(X.TVCM)*2
        )
        TVCM_m = trial.suggest_float(
            "TVCM_m",
            np.min(X.TVCM), np.max(X.TVCM)
        )
        TVCM_b = trial.suggest_float(
            "TVCM_b",
            0, 10
        )
        TVCM_c = trial.suggest_float(
            "TVCM_c",
            0, 1
        )
        
        TVCM_Saturation = pipe_logit_Saturation(
            K=TVCM_K,
            m=TVCM_m,
            b=TVCM_b,
            c=TVCM_c
        )
        
    else:
        
        TVCM_K = trial.suggest_float(
            "TVCM_K", 
            np.min(X.TVCM), 
            np.max(X.TVCM)*2
        )
        TVCM_m = trial.suggest_float(
            "TVCM_m", 
            np.min(X.TVCM), 
            np.max(X.TVCM)
        )
        TVCM_b = trial.suggest_float(
            "TVCM_b", 
            0, 1
        )
        TVCM_c = trial.suggest_float(
            "TVCM_c", 
            0, 1
        )     
        
        TVCM_Saturation = pipe_gom_Saturation(
            K=TVCM_K,
            m=TVCM_m,
            b=TVCM_b,
            c=TVCM_c
        )
        
    ##TVCM Carryover
    
    if TVCM_Carryover_func == 'simple':
        
        TVCM_rate = trial.suggest_float(
            "TVCM_rate", 
            0,1
        )
        TVCM_length = trial.suggest_int(
            "TVCM_length", 
            1,6
        )
        
        TVCM_Carryover = pipe_simple_Carryover(
            length=TVCM_length,
            rate=TVCM_rate
        )
        
    else:
        
        TVCM_rate = trial.suggest_float(
            "TVCM_rate", 
            0, 1
        )
        TVCM_length = trial.suggest_int(
            "TVCM_length",
            1,6
        )
        TVCM_peak = trial.suggest_int(
            "TVCM_peak",
            0,2
        )
        
        TVCM_Carryover = pipe_peak_Carryover(
            length=TVCM_length, 
            rate=TVCM_rate,
            peak=TVCM_peak
        )   
    #Newspaper
    
    ##Newspaper Adstock func
    
    Newspaper_Saturation_func = trial.suggest_categorical(
        "Newspaper_Saturation_func",
        ["exp", "logit","gom"]
    )
    
    Newspaper_Carryover_func = trial.suggest_categorical(
        "Newspaper_Carryover_func",
        ["simple", "peak"]
    )
    
    ##Newspaper_Saturation
    
    if Newspaper_Saturation_func == 'exp':
        
        Newspaper_a = trial.suggest_float(
            "Newspaper_a", 
            0, 0.01
        )
        
        Newspaper_Saturation = pipe_exp_Saturation(a=Newspaper_a)
        
    elif Newspaper_Saturation_func == 'logit':
        
        Newspaper_K = trial.suggest_float(
            "Newspaper_K",
            np.min(X.Newspaper), np.max(X.Newspaper)*2
        )
        Newspaper_m = trial.suggest_float(
            "Newspaper_m", 
            np.min(X.Newspaper), np.max(X.Newspaper)
        )
        Newspaper_b = trial.suggest_float(
            "Newspaper_b", 
            0, 10
        )
        Newspaper_c = trial.suggest_float(
            "Newspaper_c", 
            0, 1
        )
        
        Newspaper_Saturation = pipe_logit_Saturation(
            K=Newspaper_K,
            m=Newspaper_m,
            b=Newspaper_b,
            c=Newspaper_c
        )
        
    else:
        
        Newspaper_K = trial.suggest_float(
            "Newspaper_K", 
            np.min(X.Newspaper), np.max(X.Newspaper)*2
        )
        Newspaper_m = trial.suggest_float(
            "Newspaper_m", 
            np.min(X.Newspaper), np.max(X.Newspaper)
        )
        Newspaper_b = trial.suggest_float(
            "Newspaper_b", 
            0, 1
        )
        Newspaper_c = trial.suggest_float(
            "Newspaper_c", 
            0, 1
        )     
        
        Newspaper_Saturation = pipe_gom_Saturation(
            K=Newspaper_K,
            m=Newspaper_m,
            b=Newspaper_b,
            c=Newspaper_c
        )
        
    ##Newspaper Carryover
    
    if Newspaper_Carryover_func == 'simple':
        
        Newspaper_rate = trial.suggest_float(
            "Newspaper_rate", 
            0,1
        )
        Newspaper_length = trial.suggest_int(
            "Newspaper_length", 
            1,6
        )
        
        Newspaper_Carryover = pipe_simple_Carryover(
            length=Newspaper_length,
            rate=Newspaper_rate
        )
        
    else:
        
        Newspaper_rate = trial.suggest_float(
            "Newspaper_rate", 
            0, 1
        )
        Newspaper_length = trial.suggest_int(
            "Newspaper_length",
            1,6
        )
        Newspaper_peak = trial.suggest_int(
            "Newspaper_peak",
            0,2
        )
        
        Newspaper_Carryover = pipe_peak_Carryover(
            length=Newspaper_length,
            rate=Newspaper_rate,
            peak=Newspaper_peak
        )
            
    #Web
    
    ##Web Adstock func
    
    Web_Saturation_func = trial.suggest_categorical(
        "Web_Saturation_func",
        ["exp", "logit","gom"]
    )
    
    Web_Carryover_func = trial.suggest_categorical(
        "Web_Carryover_func",
        ["simple", "peak"]
    )
    
    ##Web_Saturation
    
    if Web_Saturation_func == 'exp':
        
        Web_a = trial.suggest_float(
            "Web_a", 
            0, 0.01
        )
        
        Web_Saturation = pipe_exp_Saturation(a=Web_a)
        
    elif Web_Saturation_func == 'logit':
        
        Web_K = trial.suggest_float(
            "Web_K", 
            np.min(X.Web), np.max(X.Web)*2
        )
        Web_m = trial.suggest_float(
            "Web_m", 
            np.min(X.Web), np.max(X.Web)
        )
        Web_b = trial.suggest_float(
            "Web_b", 
            0, 10
        )
        Web_c = trial.suggest_float(
            "Web_c", 
            0, 1
        )
        
        Web_Saturation = pipe_logit_Saturation(
            K=Web_K,
            m=Web_m,
            b=Web_b,
            c=Web_c
        )
        
    else:
        
        Web_K = trial.suggest_float(
            "Web_K", 
            np.min(X.Web), np.max(X.Web)*2
        )
        Web_m = trial.suggest_float(
            "Web_m", 
            np.min(X.Web), np.max(X.Web)
        )
        Web_b = trial.suggest_float(
            "Web_b", 
            0, 1
        )
        Web_c = trial.suggest_float(
            "Web_c", 
            0, 1
        )     
        
        Web_Saturation = pipe_gom_Saturation(
            K=Web_K,
            m=Web_m,
            b=Web_b,
            c=Web_c
        )
        
    ##Web Carryover
    
    if Web_Carryover_func == 'simple':
        
        Web_rate = trial.suggest_float(
            "Web_rate", 
            0,1
        )
        Web_length = trial.suggest_int(
            "Web_length", 
            1,6
        )
        
        Web_Carryover = pipe_simple_Carryover(
            length=Web_length,
            rate=Web_rate
        )
        
    else:
        
        Web_rate = trial.suggest_float(
            "Web_rate", 
            0, 1
        )
        Web_length = trial.suggest_int(
            "Web_length",
            1,6
        )
        Web_peak = trial.suggest_int(
            "Web_peak",
            0,2
        )
        
        Web_Carryover = pipe_peak_Carryover(
            length=Web_length,
            rate=Web_rate,
            peak=Web_peak
        )   
    
    # パイプライン化
    
    ## 変換器（Adstock）
    adstock = ColumnTransformer(
        [
         ('TVCM_pipe', Pipeline([
             ('TVCM_carryover', TVCM_Carryover),
             ('TVCM_saturation', TVCM_Saturation)
         ]), ['TVCM']),
         ('Newspaper_pipe', Pipeline([
             ('Newspaper_carryover', Newspaper_Carryover),
             ('Newspaper_saturation', Newspaper_Saturation)
         ]), ['Newspaper']),
         ('Web_pipe', Pipeline([
             ('Web_carryover', Web_Carryover),
             ('Web_saturation', Web_Saturation)
         ]), ['Web']),
        ],
        remainder='passthrough'
    )
    ## 説明変数の変換（adstock）→線形回帰モデル（regression）
    MMM_pipe = Pipeline([
        ('adstock', adstock),
        ('SS', StandardScaler()),
        ('PCA', PCA(n_components)),
        ('regression', LinearRegression())
    ])
        
    #CVによる評価
    score = cross_val_score(
        MMM_pipe,
        X,
        y,
        cv=TimeSeriesSplit()
    )
    accuracy = score.mean()
    return accuracy

- **Pipelineのハイパーパラメータを探索**

In [ ]:
# 目的関数の最適化を実行する
study = optuna.create_study(direction="maximize")
study.optimize(objective,
               n_trials=10000,
               show_progress_bar=True)

In [ ]:
# 探索結果
study.best_params

- **パイプライン構築**の、**飽和モデル**と**ラグ効果の2つの変換器クラス**を定義
- **Pipeline用変換器**

In [ ]:
## 飽和モデル

class Saturation(BaseEstimator, TransformerMixin):
    
    # 初期化
    def __init__(self,func='exp',a=1.0,K=1.0,m=100.0,b=1.0, c=0.1):
        self.func = func
        self.a = a
        self.K = K
        self.m = m
        self.b = b
        self.c = c
    
    # 学習
    def fit(self, X, y=None):
        return self
    
    # 処理（入力→出力）
    def transform(self, X):
        if self.func == 'exp':
            X_ = exp_Saturation(X,
                                self.a
                               ).reshape(-1,1)
        elif self.func == "logit":
            X_ = logit_Saturation(X,
                                  self.K, 
                                  self.b,
                                  self.c, 
                                  self.m
                                 ).reshape(-1,1)
        else:
            X_ = gom_Saturation(X,
                                self.K, 
                                self.b, 
                                self.c, 
                                self.m
                               ).reshape(-1,1)
        return X_

In [ ]:
## ラグ効果モデル
class Carryover(BaseEstimator, TransformerMixin):
    # 初期化
    def __init__(self,func='simple',length=4, peak=1, rate=0.5):
        self.func = func
        self.length = length
        self.peak = peak
        self.rate = rate
        
    # 学習        
    def fit(self, X, y=None):
        return self
    # 処理（入力→出力）
    def transform(self, X):
        if self.func == 'peak':
            X_ = peak_Carryover(X, 
                                self.length, 
                                self.peak, 
                                self.rate)  
        else:
            X_ = simple_Carryover(X, 
                                  self.rate, 
                                  self.length)
            
        return X_

- この**2つの変換器でアドストック（Ad Stock）**を表現します。これらの**変換器と学習器（PCRモデル）をつなげたパイプラインを構築**
- **Pipeline**

In [ ]:
## 説明変数の変換部分（adstock）の定義
adstock = ColumnTransformer(
    [
     ('TVCM_pipe', Pipeline([
                           ('carryover', Carryover()),
                           ('saturation', Saturation())
     ]), ['TVCM']),
     ('Newspaper_pipe', Pipeline([
                           ('carryover', Carryover()),
                           ('saturation', Saturation())
     ]), ['Newspaper']),
     ('Web_pipe', Pipeline([
                           ('carryover', Carryover()),
                           ('saturation', Saturation())
     ]), ['Web']),
    ],
    remainder='passthrough'
)


## 説明変数の変換（adstock）→線形回帰モデル（regression）
MMM_pipe = Pipeline([
    ('adstock', adstock),
    ('SS', StandardScaler()),
    ('PCA', PCA()),
    ('regression', LinearRegression())
])


## パイプラインの確認
set_config(display='diagram')   
MMM_pipe

- **最適ハイパーパラメータで学習**

In [ ]:
# 最適ハイパーパラメータ
best_params={
 'PCA__n_components': 
    study.best_params.get('n_components'),
 'adstock__TVCM_pipe__carryover__func': 
    study.best_params.get('TVCM_Carryover_func'),
 'adstock__TVCM_pipe__carryover__length': 
    study.best_params.get('TVCM_length'),
 'adstock__TVCM_pipe__carryover__peak': 
    study.best_params.get('TVCM_peak'),
 'adstock__TVCM_pipe__carryover__rate': 
    study.best_params.get('TVCM_rate'),
 'adstock__TVCM_pipe__saturation__func': 
    study.best_params.get('TVCM_Saturation_func'),
 'adstock__TVCM_pipe__saturation__a': 
    study.best_params.get('TVCM_a'),
 'adstock__TVCM_pipe__saturation__K': 
    study.best_params.get('TVCM_K'),
 'adstock__TVCM_pipe__saturation__m': 
    study.best_params.get('TVCM_m'),
 'adstock__TVCM_pipe__saturation__b': 
    study.best_params.get('TVCM_b'),
 'adstock__TVCM_pipe__saturation__c': 
    study.best_params.get('TVCM_c'),
 'adstock__Newspaper_pipe__carryover__func': 
    study.best_params.get('Newspaper_Carryover_func'),
 'adstock__Newspaper_pipe__carryover__length': 
    study.best_params.get('Newspaper_length'),
 'adstock__Newspaper_pipe__carryover__peak': 
    study.best_params.get('Newspaper_peak'),
 'adstock__Newspaper_pipe__carryover__rate': 
    study.best_params.get('Newspaper_rate'),
 'adstock__Newspaper_pipe__saturation__func': 
    study.best_params.get('Newspaper_Saturation_func'),
 'adstock__Newspaper_pipe__saturation__a': 
    study.best_params.get('Newspaper_a'),
 'adstock__Newspaper_pipe__saturation__K': 
    study.best_params.get('Newspaper_K'),
 'adstock__Newspaper_pipe__saturation__m': 
    study.best_params.get('Newspaper_m'),
 'adstock__Newspaper_pipe__saturation__b': 
    study.best_params.get('Newspaper_b'),
 'adstock__Newspaper_pipe__saturation__c': 
    study.best_params.get('Newspaper_c'),
 'adstock__Web_pipe__carryover__func':
    study.best_params.get('Web_Carryover_func'),
 'adstock__Web_pipe__carryover__length': 
    study.best_params.get('Web_length'),
 'adstock__Web_pipe__carryover__peak': 
    study.best_params.get('Web_peak'),
 'adstock__Web_pipe__carryover__rate': 
    study.best_params.get('Web_rate'),
 'adstock__Web_pipe__saturation__func': 
    study.best_params.get('Web_Saturation_func'),
 'adstock__Web_pipe__saturation__a': 
    study.best_params.get('Web_a'),
 'adstock__Web_pipe__saturation__K': 
    study.best_params.get('Web_K'),
 'adstock__Web_pipe__saturation__m': 
    study.best_params.get('Web_m'),
 'adstock__Web_pipe__saturation__b': 
    study.best_params.get('Web_b'),
 'adstock__Web_pipe__saturation__c': 
    study.best_params.get('Web_c'),
}

# パイプラインのインスタンス
MMM_pipe_best = MMM_pipe.set_params(**best_params)

# 全データで学習
MMM_pipe_best.fit(X, y)

# R2（決定係数）
MMM_pipe_best.score(X, y)

- **予測の実施**

In [ ]:
# 予測の実施

# 目的変数y（売上）の予測
pred = pd.DataFrame(MMM_pipe_best.predict(X),
                    index=X.index,
                    columns=['y'])

# 各媒体による売上の予測
## 値がすべて0の説明変数
X_ = X.copy()
X_.iloc[:,:]=0

## Base
pred['Base'] = MMM_pipe_best.predict(X_)

## 媒体
for i in range(len(X.columns)):
    X_.iloc[:,:]=0
    X_.iloc[:,i]=X.iloc[:,i]
    pred[X.columns[i]] = MMM_pipe_best.predict(X_) - pred['Base']
    
print(pred) #確認

- **貢献度とマーケティングROIの算定**

In [ ]:
# 貢献度の算定

# 予測値の補正
correction_factor = y.div(pred['y'], axis=0)   #補正係数
pred_adj = pred.mul(correction_factor, axis=0) #補正後の予測値

# 各媒体の貢献度だけ抽出
contribution = pred_adj[['Base', 'Web', 'Newspaper', 'TVCM']]
print(contribution.head())

# グラフ化
ax = (contribution
      .plot.area(
          ylabel='Sales',
          xlabel='Week')
     )
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[::-1], labels[::-1])

In [ ]:
# 貢献度の構成比

# 媒体別の貢献度の合計
contribution_sum = contribution.sum(axis=0)

# 集計結果の出力
print('売上貢献度（円）:\n', 
      contribution_sum, 
      sep=''
     )
print()
print('売上貢献度（構成比）:\n', 
      contribution_sum/contribution_sum.sum(), 
      sep=''
     )

# グラフ化
contribution_sum.plot.pie(fontsize=24)

- **マーケティングROIの算定**

In [ ]:
# マーケティングROIの算定

# 各媒体のコストの合計
cost_sum = X.sum(axis=0)

# 各媒体のROIの計算
ROI = (contribution_sum.drop('Base', axis=0) - cost_sum)/cost_sum
print('ROI:\n', ROI, sep='')

# グラフ化
ROI.plot.bar(fontsize=24)

***
<a id="PLS"></a>
## <p>**部分的最小2乗回帰(PLS)**</p>
- **PLS回帰**とは、部分的最小2乗回帰で、主成分分析回帰PCRとの違い
    - **PCR** : 主成分が、主成分の分散が最大になるように作成
    - **PLS** : 主成分が、目的変数Yと主成分の共分散が最大になるように作成
- **PLS回帰**の違い
    - **主成分回帰(PCR)**も**PLS回帰**も、**主成分が作られる**ことは同じ
    - **主成分回帰(PCR)**が説明変数だけで作られるのに対し、**PLS回帰**は目的変数との関係性も考慮して作られる
- [**PLS回帰のAdStockと最適な組み合わせ**](#AdStock_1)

- **ハイパーパラメーター：主成分の数**
    - **PLS回帰**には、**主成分回帰（PCR）**と同様にハイパーパラメータとして**主成分の数**というものがあり、モデル構築者が与える必要がある

In [ ]:
import optuna
import numpy as np
import pandas as pd

from scipy.signal import convolve2d

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer

# PLS
from sklearn.cross_decomposition import PLSRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
from optuna.distributions import IntUniformDistribution, UniformDistribution, IntDistribution, FloatDistribution
from sklearn import set_config

# Optuna
from optuna.integration import OptunaSearchCV

# Plot
import matplotlib.pyplot as plt
plt.style.use('ggplot') #グラフスタイル
plt.rcParams['figure.figsize'] = [12, 9] # グラフサイズ

In [ ]:
# データセット読み込み
url = 'https://www.salesanalytics.co.jp/4zdt'
df = pd.read_csv(url,
                 parse_dates=['Week'],
                 index_col='Week'
                )
# データ確認
print(df.info()) #変数の情報
print(df.head()) #データの一部
# 説明変数Xと目的変数yに分解
X = df.drop(columns=['Sales'])
y = df['Sales']

- Hyperparams
- **PLSの最適な主成分の数の値を探索**

In [ ]:
# PLS回帰のインスタンス生成
MMM=PLSRegression()

# 探索するハイパーパラメータ範囲の設定
params = {
 'n_components':IntUniformDistribution(1, 3),
}

# ハイパーパラメータ探索の設定
optuna_search = OptunaSearchCV(
    estimator=MMM,
    param_distributions=params,
    n_trials=1000,
    cv=TimeSeriesSplit(),
    random_state=123,
)

# 探索実施
optuna_search.fit(X, y)

# 探索結果
optuna_search.best_params_

- **最適ハイパーパラメータで学習**

In [ ]:
# パイプラインのインスタンス
MMM_best = MMM.set_params(**optuna_search.best_params_)

# 全データで学習
MMM_best.fit(X, y)

# R2（決定係数）
MMM_best.score(X, y)

- **予測の実施**

In [ ]:
# 予測の実施

# 目的変数y（売上）の予測
pred = pd.DataFrame(MMM_best.predict(X),
                    index=X.index,
                    columns=['y'])

# 各媒体による売上の予測
## 値がすべて0の説明変数
X_ = X.copy()
X_.iloc[:,:]=0

## Base
pred['Base'] = MMM_best.predict(X_)

## 媒体
for i in range(len(X.columns)):
    X_.iloc[:,:]=0
    X_.iloc[:,i]=X.iloc[:,i]
    pred[X.columns[i]] = MMM_best.predict(X_)[:,0] - pred['Base']
    
print(pred) #確認

- **貢献度とマーケティングROIの算定**

In [ ]:
# 貢献度の算定

# 予測値の補正
correction_factor = y.div(pred['y'], axis=0)   #補正係数
pred_adj = pred.mul(correction_factor, axis=0) #補正後の予測値

# 各媒体の貢献度だけ抽出
contribution = pred_adj[['Base', 'Web', 'Newspaper', 'TVCM']]
print(contribution.head()) #確認

# グラフ化
ax = (contribution
      .plot.area(
          ylabel='Sales',
          xlabel='Week')
     )
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[::-1], labels[::-1])

- **貢献度の構成比の算出**

In [ ]:
# 貢献度の構成比

# 媒体別の貢献度の合計
contribution_sum = contribution.sum(axis=0)

# 集計結果の出力
print('売上貢献度（円）:\n', 
      contribution_sum, 
      sep=''
     )
print()
print('売上貢献度（構成比）:\n', 
      contribution_sum/contribution_sum.sum(), 
      sep=''
     )

# グラフ化
contribution_sum.plot.pie(fontsize=24)

- **マーケティングROIの算定**

In [ ]:
# マーケティングROIの算定

# 各媒体のコストの合計
cost_sum = X.sum(axis=0)

# 各媒体のROIの計算
ROI = (contribution_sum.drop('Base', axis=0) - cost_sum)/cost_sum
print('ROI:\n', ROI, sep='') #確認

# グラフ化
ROI.plot.bar(fontsize=24)

***
<a id="AdStock_1"></a>
## <p>**PLSの最適な組み合わせ**</p>
- **ラグ効果モデル**
    - 定率減少型ラグ効果モデル（simple_Carryover）
    - ピーク可変型ラグ効果モデル（peak_Carryover）
- **飽和モデル**
    - 指数型飽和モデル（exp_Saturation）
    - ロジスティック型飽和モデル（logit_Saturation）
    - ゴンペルツ型飽和モデル（gom_Saturation）

In [ ]:
# 飽和モデル

### 指数型飽和モデル（exp_Saturation）
def exp_Saturation(X: np.ndarray,a):
    return 1 - np.exp(-a*X)

### ロジスティック型飽和モデル（logit_Saturation）
def logit_Saturation(X: np.ndarray,K,b,c,m):
    return K/(1+b*np.exp(-c*(X-m)))

### ゴンペルツ型飽和モデル（gom_Saturation）
def gom_Saturation(X: np.ndarray,K,b,c,m):
    return K*(b**np.exp(-c*(X-m)))


In [ ]:
## 変換器

### 指数型飽和モデル（exp_Saturation）
class pipe_exp_Saturation(BaseEstimator, TransformerMixin):
    
    # 初期化
    def __init__(self,a=1.0):
        self.a = a
    
    # 学習
    def fit(self, X, y=None):
        return self
    
    # 処理（入力→出力）
    def transform(self, X):
        X_ = exp_Saturation(X,
                            self.a
                           ).reshape(-1,1)
        return X_

    
### ロジスティック型飽和モデル（logit_Saturation）
class pipe_logit_Saturation(BaseEstimator, TransformerMixin):
    
    # 初期化
    def __init__(self,K=1.0,m=100.0,b=1.0, c=0.1):
        self.K = K
        self.m = m
        self.b = b
        self.c = c
    
    # 学習
    def fit(self, X, y=None):
        return self
    
    # 処理（入力→出力）
    def transform(self, X):
        X_ = logit_Saturation(X,
                              self.K, 
                              self.b, 
                              self.c, 
                              self.m
                             ).reshape(-1,1)
        return X_
    
    
### ゴンペルツ型飽和モデル（gom_Saturation）
class pipe_gom_Saturation(BaseEstimator, TransformerMixin):
    
    # 初期化
    def __init__(self,K=1.0,m=100.0,b=1.0,c=0.1):
        self.K = K
        self.m = m
        self.b = b
        self.c = c
    
    # 学習
    def fit(self, X, y=None):
        return self
    
    # 処理（入力→出力）
    def transform(self, X):
        X_ = gom_Saturation(X,
                            self.K, 
                            self.b, 
                            self.c, 
                            self.m
                           ).reshape(-1,1)
        return X_

In [ ]:
# ラグ効果モデル


### 定率減少型ラグ効果モデル（simple_Carryover）
def simple_Carryover(X: np.ndarray, rate, length):
    
    filter = (
        rate ** np.arange(length)
    ).reshape(-1, 1) 
    
    convolution = convolve2d(X, filter)
    
    if length > 1 : convolution = convolution[: -(length-1)]
        
    return convolution


### ピーク可変型ラグ効果モデル（peak_Carryover）
def peak_Carryover(X: np.ndarray, length, peak, rate):
    X = np.append(np.zeros(length-1), X)
    
    Ws = np.zeros(length)
    
    for l in range(length):
        W = rate**((l-peak)**2)
        Ws[length-1-l] = W
    
    carryover_X = []
    
    for i in range(length-1, len(X)):
        X_array = X[i-length+1:i+1]
        Xi = sum(X_array * Ws)/sum(Ws)
        carryover_X.append(Xi)
        
    return np.array(carryover_X)

In [ ]:
## 変換器

### 定率減少型ラグ効果モデル　※ピークが広告など投入時（simple_Carryover）
class pipe_simple_Carryover(BaseEstimator, TransformerMixin):
    # 初期化
    def __init__(self,length=4,rate=0.5):
        self.length = length
        self.rate = rate
        
    # 学習        
    def fit(self, X, y=None):
        return self
    # 処理（入力→出力）
    def transform(self, X):
        X_ = simple_Carryover(X, 
                              self.rate, 
                              self.length
                             )
        return X_

    
### ピーク可変型ラグ効果モデル
class pipe_peak_Carryover(BaseEstimator, TransformerMixin):
    # 初期化
    def __init__(self,length=4,peak=1,rate=0.5):
        self.length = length
        self.peak = peak
        self.rate = rate
        
    # 学習        
    def fit(self, X, y=None):
        return self
    # 処理（入力→出力）
    def transform(self, X):
        X_ = peak_Carryover(X, 
                            self.length, 
                            self.peak, 
                            self.rate
                           ) 
        return X_

In [ ]:
# 目的関数の設定
def objective(trial):
    
    #PCA
    n_components = trial.suggest_int(
        "n_components",
        1, 3
    )
    
    #TVCM
    
    ##TVCM Adstock func
    
    TVCM_Saturation_func = trial.suggest_categorical(
        "TVCM_Saturation_func",
        ["exp", "logit","gom"]
    )
    
    TVCM_Carryover_func = trial.suggest_categorical(
        "TVCM_Carryover_func",
        ["simple", "peak"]
    )
    
    ##TVCM_Saturation
    
    if TVCM_Saturation_func == 'exp':
        
        TVCM_a = trial.suggest_float(
            "TVCM_a", 
            0, 0.01
        )
        
        TVCM_Saturation = pipe_exp_Saturation(a=TVCM_a)
        
    elif TVCM_Saturation_func == 'logit':
        
        TVCM_K = trial.suggest_float(
            "TVCM_K", 
            np.min(X.TVCM), np.max(X.TVCM)*2
        )
        TVCM_m = trial.suggest_float(
            "TVCM_m",
            np.min(X.TVCM), np.max(X.TVCM)
        )
        TVCM_b = trial.suggest_float(
            "TVCM_b",
            0, 10
        )
        TVCM_c = trial.suggest_float(
            "TVCM_c",
            0, 1
        )
        
        TVCM_Saturation = pipe_logit_Saturation(
            K=TVCM_K,
            m=TVCM_m,
            b=TVCM_b,
            c=TVCM_c
        )
        
    else:
        
        TVCM_K = trial.suggest_float(
            "TVCM_K", 
            np.min(X.TVCM), 
            np.max(X.TVCM)*2
        )
        TVCM_m = trial.suggest_float(
            "TVCM_m", 
            np.min(X.TVCM), 
            np.max(X.TVCM)
        )
        TVCM_b = trial.suggest_float(
            "TVCM_b", 
            0, 1
        )
        TVCM_c = trial.suggest_float(
            "TVCM_c", 
            0, 1
        )     
        
        TVCM_Saturation = pipe_gom_Saturation(
            K=TVCM_K,
            m=TVCM_m,
            b=TVCM_b,
            c=TVCM_c
        )
        
    ##TVCM Carryover
    
    if TVCM_Carryover_func == 'simple':
        
        TVCM_rate = trial.suggest_float(
            "TVCM_rate", 
            0,1
        )
        TVCM_length = trial.suggest_int(
            "TVCM_length", 
            1,6
        )
        
        TVCM_Carryover = pipe_simple_Carryover(
            length=TVCM_length,
            rate=TVCM_rate
        )
        
    else:
        
        TVCM_rate = trial.suggest_float(
            "TVCM_rate", 
            0, 1
        )
        TVCM_length = trial.suggest_int(
            "TVCM_length",
            1,6
        )
        TVCM_peak = trial.suggest_int(
            "TVCM_peak",
            0,2
        )
        
        TVCM_Carryover = pipe_peak_Carryover(
            length=TVCM_length, 
            rate=TVCM_rate,
            peak=TVCM_peak
        )   
    #Newspaper
    
    ##Newspaper Adstock func
    
    Newspaper_Saturation_func = trial.suggest_categorical(
        "Newspaper_Saturation_func",
        ["exp", "logit","gom"]
    )
    
    Newspaper_Carryover_func = trial.suggest_categorical(
        "Newspaper_Carryover_func",
        ["simple", "peak"]
    )
    
    ##Newspaper_Saturation
    
    if Newspaper_Saturation_func == 'exp':
        
        Newspaper_a = trial.suggest_float(
            "Newspaper_a", 
            0, 0.01
        )
        
        Newspaper_Saturation = pipe_exp_Saturation(a=Newspaper_a)
        
    elif Newspaper_Saturation_func == 'logit':
        
        Newspaper_K = trial.suggest_float(
            "Newspaper_K",
            np.min(X.Newspaper), np.max(X.Newspaper)*2
        )
        Newspaper_m = trial.suggest_float(
            "Newspaper_m", 
            np.min(X.Newspaper), np.max(X.Newspaper)
        )
        Newspaper_b = trial.suggest_float(
            "Newspaper_b", 
            0, 10
        )
        Newspaper_c = trial.suggest_float(
            "Newspaper_c", 
            0, 1
        )
        
        Newspaper_Saturation = pipe_logit_Saturation(
            K=Newspaper_K,
            m=Newspaper_m,
            b=Newspaper_b,
            c=Newspaper_c
        )
        
    else:
        
        Newspaper_K = trial.suggest_float(
            "Newspaper_K", 
            np.min(X.Newspaper), np.max(X.Newspaper)*2
        )
        Newspaper_m = trial.suggest_float(
            "Newspaper_m", 
            np.min(X.Newspaper), np.max(X.Newspaper)
        )
        Newspaper_b = trial.suggest_float(
            "Newspaper_b", 
            0, 1
        )
        Newspaper_c = trial.suggest_float(
            "Newspaper_c", 
            0, 1
        )     
        
        Newspaper_Saturation = pipe_gom_Saturation(
            K=Newspaper_K,
            m=Newspaper_m,
            b=Newspaper_b,
            c=Newspaper_c
        )
        
    ##Newspaper Carryover
    
    if Newspaper_Carryover_func == 'simple':
        
        Newspaper_rate = trial.suggest_float(
            "Newspaper_rate", 
            0,1
        )
        Newspaper_length = trial.suggest_int(
            "Newspaper_length", 
            1,6
        )
        
        Newspaper_Carryover = pipe_simple_Carryover(
            length=Newspaper_length,
            rate=Newspaper_rate
        )
        
    else:
        
        Newspaper_rate = trial.suggest_float(
            "Newspaper_rate", 
            0, 1
        )
        Newspaper_length = trial.suggest_int(
            "Newspaper_length",
            1,6
        )
        Newspaper_peak = trial.suggest_int(
            "Newspaper_peak",
            0,2
        )
        
        Newspaper_Carryover = pipe_peak_Carryover(
            length=Newspaper_length,
            rate=Newspaper_rate,
            peak=Newspaper_peak
        )
            
    #Web
    
    ##Web Adstock func
    
    Web_Saturation_func = trial.suggest_categorical(
        "Web_Saturation_func",
        ["exp", "logit","gom"]
    )
    
    Web_Carryover_func = trial.suggest_categorical(
        "Web_Carryover_func",
        ["simple", "peak"]
    )
    
    ##Web_Saturation
    
    if Web_Saturation_func == 'exp':
        
        Web_a = trial.suggest_float(
            "Web_a", 
            0, 0.01
        )
        
        Web_Saturation = pipe_exp_Saturation(a=Web_a)
        
    elif Web_Saturation_func == 'logit':
        
        Web_K = trial.suggest_float(
            "Web_K", 
            np.min(X.Web), np.max(X.Web)*2
        )
        Web_m = trial.suggest_float(
            "Web_m", 
            np.min(X.Web), np.max(X.Web)
        )
        Web_b = trial.suggest_float(
            "Web_b", 
            0, 10
        )
        Web_c = trial.suggest_float(
            "Web_c", 
            0, 1
        )
        
        Web_Saturation = pipe_logit_Saturation(
            K=Web_K,
            m=Web_m,
            b=Web_b,
            c=Web_c
        )
        
    else:
        
        Web_K = trial.suggest_float(
            "Web_K", 
            np.min(X.Web), np.max(X.Web)*2
        )
        Web_m = trial.suggest_float(
            "Web_m", 
            np.min(X.Web), np.max(X.Web)
        )
        Web_b = trial.suggest_float(
            "Web_b", 
            0, 1
        )
        Web_c = trial.suggest_float(
            "Web_c", 
            0, 1
        )     
        
        Web_Saturation = pipe_gom_Saturation(
            K=Web_K,
            m=Web_m,
            b=Web_b,
            c=Web_c
        )
        
    ##Web Carryover
    
    if Web_Carryover_func == 'simple':
        
        Web_rate = trial.suggest_float(
            "Web_rate", 
            0,1
        )
        Web_length = trial.suggest_int(
            "Web_length", 
            1,6
        )
        
        Web_Carryover = pipe_simple_Carryover(
            length=Web_length,
            rate=Web_rate
        )
        
    else:
        
        Web_rate = trial.suggest_float(
            "Web_rate", 
            0, 1
        )
        Web_length = trial.suggest_int(
            "Web_length",
            1,6
        )
        Web_peak = trial.suggest_int(
            "Web_peak",
            0,2
        )
        
        Web_Carryover = pipe_peak_Carryover(
            length=Web_length,
            rate=Web_rate,
            peak=Web_peak
        )   
    
    # パイプライン化
    
    ## 変換器（Adstock）
    adstock = ColumnTransformer(
        [
         ('TVCM_pipe', Pipeline([
             ('TVCM_carryover', TVCM_Carryover),
             ('TVCM_saturation', TVCM_Saturation)
         ]), ['TVCM']),
         ('Newspaper_pipe', Pipeline([
             ('Newspaper_carryover', Newspaper_Carryover),
             ('Newspaper_saturation', Newspaper_Saturation)
         ]), ['Newspaper']),
         ('Web_pipe', Pipeline([
             ('Web_carryover', Web_Carryover),
             ('Web_saturation', Web_Saturation)
         ]), ['Web']),
        ],
        remainder='passthrough'
    )
    
    ## 説明変数の変換（adstock）→PLS回帰モデル
    MMM_pipe = Pipeline([
        ('adstock', adstock),
        ('PLS', PLSRegression(n_components))
    ])
        
    #CVによる評価
    score = cross_val_score(
        MMM_pipe,
        X,
        y,
        cv=TimeSeriesSplit()
    )
    accuracy = score.mean()
    return accuracy

In [ ]:
# 目的関数の最適化を実行する
study = optuna.create_study(direction="maximize")
study.optimize(objective,
               n_trials=10000,
               show_progress_bar=True)

In [ ]:
# 探索結果
study.best_params

- **パイプライン構築**
- パイプライン用の、**飽和モデル**と**ラグ効果の2つの変換器クラス**を定義

In [ ]:
## 飽和モデル
class Saturation(BaseEstimator, TransformerMixin):
    
    # 初期化
    def __init__(self,func='exp',a=1.0,K=1.0,m=100.0,b=1.0, c=0.1):
        self.func = func
        self.a = a
        self.K = K
        self.m = m
        self.b = b
        self.c = c
    
    # 学習
    def fit(self, X, y=None):
        return self
    
    # 処理（入力→出力）
    def transform(self, X):
        if self.func == 'exp':
            X_ = exp_Saturation(X,
                                self.a
                               ).reshape(-1,1)
        elif self.func == "logit":
            X_ = logit_Saturation(X,
                                  self.K, 
                                  self.b,
                                  self.c, 
                                  self.m
                                 ).reshape(-1,1)
        else:
            X_ = gom_Saturation(X,
                                self.K, 
                                self.b, 
                                self.c, 
                                self.m
                               ).reshape(-1,1)
        return X_

In [ ]:
## ラグ効果モデル
class Carryover(BaseEstimator, TransformerMixin):
    # 初期化
    def __init__(self,func='simple',length=4, peak=1, rate=0.5):
        self.func = func
        self.length = length
        self.peak = peak
        self.rate = rate
        
    # 学習        
    def fit(self, X, y=None):
        return self
    # 処理（入力→出力）
    def transform(self, X):
        if self.func == 'peak':
            X_ = peak_Carryover(X, 
                                self.length, 
                                self.peak, 
                                self.rate)  
        else:
            X_ = simple_Carryover(X, 
                                  self.rate, 
                                  self.length)
            
        return X_

- **2つの変換器でアドストック（Ad Stock）**を表現します。これらの**変換器と学習器（PLS回帰モデル）をつなげたパイプラインを構築**

In [ ]:
## 説明変数の変換部分（adstock）の定義
adstock = ColumnTransformer(
    [
     ('TVCM_pipe', Pipeline([
                           ('carryover', Carryover()),
                           ('saturation', Saturation())
     ]), ['TVCM']),
     ('Newspaper_pipe', Pipeline([
                           ('carryover', Carryover()),
                           ('saturation', Saturation())
     ]), ['Newspaper']),
     ('Web_pipe', Pipeline([
                           ('carryover', Carryover()),
                           ('saturation', Saturation())
     ]), ['Web']),
    ],
    remainder='passthrough'
)

## 説明変数の変換（adstock）→PLS回帰モデル（PLS）
MMM_pipe = Pipeline([
    ('adstock', adstock),
    ('PLS', PLSRegression())
])

## パイプラインの確認
set_config(display='diagram')   
MMM_pipe

- **最適ハイパーパラメータで学習**

In [ ]:
# 最適ハイパーパラメータ
best_params={
 'PLS__n_components': 
    study.best_params.get('n_components'),
 'adstock__TVCM_pipe__carryover__func': 
    study.best_params.get('TVCM_Carryover_func'),
 'adstock__TVCM_pipe__carryover__length': 
    study.best_params.get('TVCM_length'),
 'adstock__TVCM_pipe__carryover__peak': 
    study.best_params.get('TVCM_peak'),
 'adstock__TVCM_pipe__carryover__rate': 
    study.best_params.get('TVCM_rate'),
 'adstock__TVCM_pipe__saturation__func': 
    study.best_params.get('TVCM_Saturation_func'),
 'adstock__TVCM_pipe__saturation__a': 
    study.best_params.get('TVCM_a'),
 'adstock__TVCM_pipe__saturation__K': 
    study.best_params.get('TVCM_K'),
 'adstock__TVCM_pipe__saturation__m': 
    study.best_params.get('TVCM_m'),
 'adstock__TVCM_pipe__saturation__b': 
    study.best_params.get('TVCM_b'),
 'adstock__TVCM_pipe__saturation__c': 
    study.best_params.get('TVCM_c'),
 'adstock__Newspaper_pipe__carryover__func': 
    study.best_params.get('Newspaper_Carryover_func'),
 'adstock__Newspaper_pipe__carryover__length': 
    study.best_params.get('Newspaper_length'),
 'adstock__Newspaper_pipe__carryover__peak': 
    study.best_params.get('Newspaper_peak'),
 'adstock__Newspaper_pipe__carryover__rate': 
    study.best_params.get('Newspaper_rate'),
 'adstock__Newspaper_pipe__saturation__func': 
    study.best_params.get('Newspaper_Saturation_func'),
 'adstock__Newspaper_pipe__saturation__a': 
    study.best_params.get('Newspaper_a'),
 'adstock__Newspaper_pipe__saturation__K': 
    study.best_params.get('Newspaper_K'),
 'adstock__Newspaper_pipe__saturation__m': 
    study.best_params.get('Newspaper_m'),
 'adstock__Newspaper_pipe__saturation__b': 
    study.best_params.get('Newspaper_b'),
 'adstock__Newspaper_pipe__saturation__c': 
    study.best_params.get('Newspaper_c'),
 'adstock__Web_pipe__carryover__func':
    study.best_params.get('Web_Carryover_func'),
 'adstock__Web_pipe__carryover__length': 
    study.best_params.get('Web_length'),
 'adstock__Web_pipe__carryover__peak': 
    study.best_params.get('Web_peak'),
 'adstock__Web_pipe__carryover__rate': 
    study.best_params.get('Web_rate'),
 'adstock__Web_pipe__saturation__func': 
    study.best_params.get('Web_Saturation_func'),
 'adstock__Web_pipe__saturation__a': 
    study.best_params.get('Web_a'),
 'adstock__Web_pipe__saturation__K': 
    study.best_params.get('Web_K'),
 'adstock__Web_pipe__saturation__m': 
    study.best_params.get('Web_m'),
 'adstock__Web_pipe__saturation__b': 
    study.best_params.get('Web_b'),
 'adstock__Web_pipe__saturation__c': 
    study.best_params.get('Web_c'),
}

# パイプラインのインスタンス
MMM_pipe_best = MMM_pipe.set_params(**best_params)

# 全データで学習
MMM_pipe_best.fit(X, y)

# R2（決定係数）
MMM_pipe_best.score(X, y)

- **予測の実施**

In [ ]:
# 予測の実施
# 
# 目的変数y（売上）の予測
pred = pd.DataFrame(MMM_pipe_best.predict(X),
                    index=X.index,
                    columns=['y'])
# 各媒体による売上の予測
## 値がすべて0の説明変数
X_ = X.copy()
X_.iloc[:,:]=0
## Base
pred['Base'] = MMM_pipe_best.predict(X_)
## 媒体
for i in range(len(X.columns)):
    X_.iloc[:,:]=0
    X_.iloc[:,i]=X.iloc[:,i]
    pred[X.columns[i]] = MMM_pipe_best.predict(X_)[:,0]-pred['Base']
    
print(pred) #確認

# グラフ化（実測値と予測値）
## 散布図を描画
plt.scatter(y,pred['y'])
plt.title('R2:'+str(round(MMM_pipe_best.score(X, y),4)))
plt.ylabel('pred')
plt.xlabel('actual')
plt.show()
## 折れ線グラフ（時系列推移）
fig, ax = plt.subplots()
ax.plot(y.index, y.values, label="actual")
ax.plot(y.index, pred['y'].values, label="pred", linestyle="dotted", lw=2) 
ax.set_ylim(0, 4e6)
plt.legend()

- **貢献度とマーケティングROIの算定**

In [ ]:
# 貢献度

# 予測値の補正
correction_factor = y.div(pred['y'], axis=0)   #補正係数
pred_adj = pred.mul(correction_factor, axis=0) #補正後の予測値

# 各媒体の貢献度だけ抽出
contribution = pred_adj[['Base', 'Web', 'Newspaper', 'TVCM']]
print(contribution.head())

# グラフ化
ax = (contribution
      .plot.area(
          ylabel='Sales',
          xlabel='Week')
     )
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[::-1], labels[::-1])

In [ ]:
# 貢献度の構成比

# 媒体別の貢献度の合計
contribution_sum = contribution.sum(axis=0)
# 集計結果の出力
print('売上貢献度（円）:\n', 
      contribution_sum, 
      sep=''
     )
print()
print('売上貢献度（構成比）:\n', 
      contribution_sum/contribution_sum.sum(), 
      sep=''
     )

# グラフ化
contribution_sum.plot.pie(fontsize=24)

In [ ]:
# マーケティングROI

# 各媒体のコストの合計
cost_sum = X.sum(axis=0)

# 各媒体のROIの計算
ROI = (contribution_sum.drop('Base', axis=0) - cost_sum)/cost_sum
print('ROI:\n', ROI, sep='') #確認

# グラフ化
ROI.plot.bar(fontsize=24)